# LGBM

In [ ]:
# =========================================================
# Blok 0 — Setup, Paths, dan Library (LightGBM only)
# =========================================================
import os, gc, math, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Install LightGBM (jika belum tersedia)
try:
    import lightgbm as lgb
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "lightgbm"])
    import lightgbm as lgb

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# ====== Paths (prioritas Kaggle sesuai permintaan) ======
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback (opsional, jika ingin jalankan di lokal):
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
np.random.seed(SEED)


In [2]:
# =========================================================
# Blok 1 — Load Data, Sort Timestamp, dan Cek Kolom
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

# Parse timestamp dan urutkan
for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

TARGET = "Turbidity"
IDCOL  = "Record number"

FEATURES_RAW = [
    "Average Water Speed",
    "Average Water Direction",
    "Chlorophyll",
    "Temperature",
    "Dissolved Oxygen",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Salinity",
    "Specific Conductance",
]

# Cek kolom wajib
missing_train = [c for c in FEATURES_RAW+[TARGET,"Timestamp",IDCOL] if c not in train.columns]
missing_test  = [c for c in FEATURES_RAW+["Timestamp",IDCOL] if c not in test.columns]
if missing_train: print("Peringatan, kolom hilang di train:", missing_train)
if missing_test:  print("Peringatan, kolom hilang di test :", missing_test)


In [3]:
# =========================================================
# Blok 2 — Fitur Waktu & Arah + Interaksi Ringan
# =========================================================
def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt) == 0:
        return 1
    median_sec = np.median(dt)
    if not np.isfinite(median_sec) or median_sec <= 0:
        return 1
    return int(max(1, round(3600.0 / median_sec)))

def add_time_harmonics(df, ts_col="Timestamp"):
    # siklus harian, mingguan, dan pasang 12.42 jam
    out = df.copy()
    s = out[ts_col].astype("int64") // 10**9
    day_sec = 24*3600
    week_sec = 7*24*3600
    tide_sec = int(round(12.42*3600))
    out["sin_day"]  = np.sin(2*np.pi * (s % day_sec) / day_sec)
    out["cos_day"]  = np.cos(2*np.pi * (s % day_sec) / day_sec)
    out["sin_week"] = np.sin(2*np.pi * (s % week_sec) / week_sec)
    out["cos_week"] = np.cos(2*np.pi * (s % week_sec) / week_sec)
    out["sin_tide"] = np.sin(2*np.pi * (s % tide_sec) / tide_sec)
    out["cos_tide"] = np.cos(2*np.pi * (s % tide_sec) / tide_sec)
    return out

def dir_to_sincos(df, dir_col="Average Water Direction", spd_col="Average Water Speed"):
    out = df.copy()
    rad = np.deg2rad(out[dir_col])
    out["dir_sin"] = np.sin(rad)
    out["dir_cos"] = np.cos(rad)
    # proyeksi arus
    out["u_comp"] = out[spd_col] * out["dir_cos"]
    out["v_comp"] = out[spd_col] * out["dir_sin"]
    return out

# Gabung train+test untuk konsistensi fitur non-target
train["__is_train__"] = 1
test["__is_train__"]  = 0
full = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

# Sanitasi ringan
full["pH"] = full["pH"].clip(lower=0, upper=14)

# Tambahkan fitur waktu & arah
full = add_time_harmonics(full, "Timestamp")
full = dir_to_sincos(full, "Average Water Direction", "Average Water Speed")

# Interaksi domain ringan
full["temp_do_sat"] = full["Temperature"] * full["Dissolved Oxygen (%Saturation)"]
full["sal_conduct_ratio"] = full["Salinity"] / full["Specific Conductance"].replace(0, np.nan)
full["sal_conduct_ratio"] = full["sal_conduct_ratio"].replace([np.inf, -np.inf], np.nan)

# Estimasi resolusi data → langkah per jam (untuk menentukan lag/rolling)
steps_per_hour = estimate_steps_per_hour(full)
steps_1h  = max(1, steps_per_hour)
steps_3h  = max(1, 3*steps_per_hour)
steps_6h  = max(1, 6*steps_per_hour)
steps_24h = max(1, 24*steps_per_hour)

EXTRA_FEATS_BASE = [
    "sin_day","cos_day","sin_week","cos_week","sin_tide","cos_tide",
    "dir_sin","dir_cos","u_comp","v_comp","temp_do_sat","sal_conduct_ratio"
]


In [4]:
# =========================================================
# Blok 3 — Imputasi Time-Aware + Lag & Rolling Tanpa Bocor
# =========================================================
def timewise_impute(df, cols):
    out = df.copy()
    out[cols] = out[cols].ffill().bfill()
    for c in cols:
        # rolling median ke belakang (no leakage)
        out[c] = out[c].fillna(out[c].rolling(window=steps_6h, min_periods=1).median())
        out[c] = out[c].fillna(out[c].median())
    return out

def add_lag_roll(df, cols, lags, rolls):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
        for w in rolls:
            # gunakan shift(1) agar hanya masa lalu
            out[f"{c}_roll{w}_mean"] = out[c].shift(1).rolling(window=w, min_periods=1).mean()
            out[f"{c}_roll{w}_std"]  = out[c].shift(1).rolling(window=w, min_periods=1).std()
            out[f"{c}_roll{w}_min"]  = out[c].shift(1).rolling(window=w, min_periods=1).min()
            out[f"{c}_roll{w}_max"]  = out[c].shift(1).rolling(window=w, min_periods=1).max()
    return out

# Kolom yang diimputasi/diturunkan lag&rolling
sensor_cols = FEATURES_RAW + ["u_comp","v_comp","temp_do_sat","sal_conduct_ratio","dir_sin","dir_cos"]
full = timewise_impute(full, sensor_cols)

# Tambahkan lag & rolling
LAGS  = [1, steps_1h, steps_3h]                  # 1-step, ~1 jam, ~3 jam
ROLLS = [steps_1h, steps_6h, steps_24h]          # ~1h, ~6h, ~24h
full = add_lag_roll(full, sensor_cols, LAGS, ROLLS)

# Daftar fitur final
lagroll_feats = [c for c in full.columns if any(s in c for s in ["_lag", "_roll"])]
FEATURES = FEATURES_RAW + EXTRA_FEATS_BASE + lagroll_feats

# Split kembali ke train/test (target tetap dari frame train awal)
train_fe = full[full["__is_train__"]==1].copy()
test_fe  = full[full["__is_train__"]==0].copy()
train_fe[TARGET] = train[TARGET].values  # align target ke urutan baru

# Drop baris train yang masih NaN setelah lag/rolling
train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

# Isi sisa NaN di test dengan median train (aman untuk inference)
test_X = test_fe[FEATURES].copy()
train_X = train_fe[FEATURES].copy()
test_X = test_X.fillna(train_X.median())
y = train_fe[TARGET].values


In [5]:
# =========================================================
# Blok 4 — TimeSeries CV (LightGBM, Early Stopping, OOF)
# =========================================================
n_splits = 5
gap = max(1, steps_1h)  # buffer ~1 jam
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

# Parameter LGBM
lgb_params = dict(
    objective="l2",               # MSE
    learning_rate=0.05,
    num_leaves=64,
    feature_fraction=0.9,
    bagging_fraction=0.8,
    bagging_freq=1,
    min_data_in_leaf=50,
    n_estimators=5000,            # besar, akan dipangkas early_stopping
    random_state=SEED,
    verbose=-1
)

oof_pred = np.zeros(len(train_fe))
best_iters = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=300, verbose=False)]
    )

    pred_va = model.predict(X_va, num_iteration=model.best_iteration_)
    oof_pred[va_idx] = np.clip(pred_va, 0, None)  # NTU tidak negatif

    fold_mse = mean_squared_error(y[va_idx], oof_pred[va_idx])
    best_iters.append(model.best_iteration_)
    print(f"Fold {fold} — best_iter={model.best_iteration_}, MSE={fold_mse:.6f}")

cv_mse = mean_squared_error(y, oof_pred)
best_iter_final = int(np.median(best_iters)) if len(best_iters)>0 else lgb_params["n_estimators"]
print(f"\nCV MSE (OOF) = {cv_mse:.6f} | median best_iter = {best_iter_final}")


Fold 1 — best_iter=3153, MSE=4.507777
Fold 2 — best_iter=347, MSE=39.994899
Fold 3 — best_iter=2, MSE=13.195891
Fold 4 — best_iter=6, MSE=8.059490
Fold 5 — best_iter=204, MSE=45.425432

CV MSE (OOF) = 19.392469 | median best_iter = 204


In [6]:
# =========================================================
# Blok 5 — Train Full Model dengan best_iter & Prediksi Test
# =========================================================
# Latih ulang full model memakai jumlah tree = median best_iter dari CV
lgb_full = lgb.LGBMRegressor(**{**lgb_params, "n_estimators": best_iter_final})
lgb_full.fit(train_X, y)

pred_test = lgb_full.predict(test_X)
pred_test = np.clip(pred_test, 0, None)  # jaga non-negatif

# Buat submission.csv (urut sesuai sample_submission)
submission = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": pred_test
})

if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))


✅ submission.csv dibuat!
   Record number  Turbidity
0          54916   3.230723
1          54917   3.343420
2          54918   3.766943
3          54919   4.195236
4          54920   4.720352
5          54921   4.866751
6          54922   4.671263
7          54923   5.292683
8          54924   5.623497
9          54925   6.091133


In [ ]:
# =========================================================
# Blok 6 — (Opsional) Penting: Cek Fitur Penting LightGBM
# =========================================================
try:
    importances = pd.DataFrame({
        "feature": train_X.columns,
        "importance": lgb_full.booster_.feature_importance(importance_type="gain")
    }).sort_values("importance", ascending=False)
    print("Top 30 feature importance (gain):")
    print(importances.head(30))
except Exception as e:
    print("Feature importance tidak tersedia:", e)


In [ ]:
# =========================================================
# Blok 7 — Simpan Model & Artefak (model, fitur, OOF, FI, submission)
# =========================================================
import json, os
from pathlib import Path
try:
    import joblib
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "joblib"])
    import joblib

OUTPUT_DIR = Path("./outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Simpan submission (di root & di outputs/)
submission.to_csv("submission.csv", index=False)
submission.to_csv(OUTPUT_DIR / "submission.csv", index=False)

# 2) Simpan model LightGBM
#    a) Booster (format text .txt, bisa dibuka/edit, kompatibel LightGBM)
lgb_full.booster_.save_model(str(OUTPUT_DIR / "lgbm_turbidity.txt"))
#    b) Sklearn wrapper (pickle .pkl via joblib, langsung .predict)
joblib.dump(lgb_full, OUTPUT_DIR / "lgbm_sklearn.pkl")

# 3) Simpan daftar fitur (urutan kolom penting untuk inference)
pd.Series(train_X.columns).to_csv(OUTPUT_DIR / "features.txt", index=False, header=False)

# 4) Simpan feature importance (gain & split)
fi = pd.DataFrame({
    "feature": train_X.columns,
    "gain":   lgb_full.booster_.feature_importance(importance_type="gain"),
    "split":  lgb_full.booster_.feature_importance(importance_type="split"),
}).sort_values("gain", ascending=False)
fi.to_csv(OUTPUT_DIR / "feature_importance.csv", index=False)

# 5) Simpan OOF (untuk audit CV)
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_pred[:len(oof_df)]
oof_df.to_csv(OUTPUT_DIR / "oof_predictions.csv", index=False)

# 6) Simpan metadata ringkas
meta = {
    "cv_mse": float(cv_mse),
    "best_iter_final": int(best_iter_final),
    "n_features": int(len(train_X.columns)),
    "seed": int(SEED),
    "steps_per_hour": int(steps_per_hour)
}
with open(OUTPUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Artefak disimpan di:", OUTPUT_DIR.resolve())
print("📄 Daftar file:", [p.name for p in OUTPUT_DIR.iterdir()])


In [8]:
# =========================================================
# Blok 9 — LGBM-only CV-Ensemble (TimeSeriesSplit)
# =========================================================
import joblib, json, gc
from pathlib import Path

ENS_DIR = Path("./outputs_cv_ensemble")
ENS_DIR.mkdir(parents=True, exist_ok=True)

cv_models = []
cv_best_iters = []
test_preds_folds = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=300, verbose=False)]
    )
    best_it = model.best_iteration_
    cv_models.append(model)
    cv_best_iters.append(int(best_it) if best_it is not None else int(lgb_params["n_estimators"]))

    # Prediksi test dengan model fold ini
    pred_te = model.predict(test_X, num_iteration=best_it)
    pred_te = np.clip(pred_te, 0, None)
    test_preds_folds.append(pred_te)

    # Simpan model per fold (booster dan joblib)
    model.booster_.save_model(str(ENS_DIR / f"lgbm_fold{fold}.txt"))
    joblib.dump(model, ENS_DIR / f"lgbm_fold{fold}.pkl")

    # FI per fold
    fi_fold = pd.DataFrame({
        "feature": train_X.columns,
        "gain":   model.booster_.feature_importance(importance_type="gain"),
        "split":  model.booster_.feature_importance(importance_type="split"),
        "fold":   fold
    }).sort_values("gain", ascending=False)
    fi_fold.to_csv(ENS_DIR / f"feature_importance_fold{fold}.csv", index=False)

    # OOF fold (untuk audit)
    oof_fold = pd.DataFrame({
        "Record number": train_fe.iloc[va_idx][IDCOL].values,
        "Timestamp":     train_fe.iloc[va_idx]["Timestamp"].values,
        TARGET:          y[va_idx],
        "oof_pred":      np.clip(model.predict(train_X.iloc[va_idx], num_iteration=best_it), 0, None)
    })
    oof_fold.to_csv(ENS_DIR / f"oof_fold{fold}.csv", index=False)

    print(f"Ensemble Fold {fold} — best_iter={best_it}, test_pred_mean={pred_te.mean():.4f}")

# Rata-rata prediksi test dari semua fold (CV-ensemble)
pred_test_cv_ens = np.mean(np.vstack(test_preds_folds), axis=0)
pred_test_cv_ens = np.clip(pred_test_cv_ens, 0, None)

# Simpan ringkasan ensemble
with open(ENS_DIR / "cv_ensemble_meta.json", "w") as f:
    json.dump({
        "n_models": len(cv_models),
        "best_iters": cv_best_iters,
        "avg_best_iter": float(np.mean(cv_best_iters)) if len(cv_best_iters)>0 else None
    }, f, indent=2)

# FI agregat (gain rata-rata across folds)
fi_all = []
for fold in range(1, len(cv_models)+1):
    fip = pd.read_csv(ENS_DIR / f"feature_importance_fold{fold}.csv")
    fi_all.append(fip[["feature", "gain"]].set_index("feature"))
fi_avg = pd.concat(fi_all, axis=1)
fi_avg.columns = [f"fold{idx+1}" for idx in range(len(cv_models))]
fi_avg["avg_gain"] = fi_avg.mean(axis=1)
fi_avg = fi_avg.sort_values("avg_gain", ascending=False).reset_index()
fi_avg.to_csv(ENS_DIR / "feature_importance_avg.csv", index=False)

gc.collect()
print("✅ CV-ensemble selesai. Model & FI per-fold tersimpan di", ENS_DIR.resolve())


Ensemble Fold 1 — best_iter=3153, test_pred_mean=3.2714
Ensemble Fold 2 — best_iter=347, test_pred_mean=9.2749
Ensemble Fold 3 — best_iter=2, test_pred_mean=4.1535
Ensemble Fold 4 — best_iter=6, test_pred_mean=4.9754
Ensemble Fold 5 — best_iter=204, test_pred_mean=7.8459
✅ CV-ensemble selesai. Model & FI per-fold tersimpan di /kaggle/working/outputs_cv_ensemble


In [9]:
# =========================================================
# Blok 10 — Buat submission dari CV-Ensemble & Simpan
# =========================================================
submission_cv = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": pred_test_cv_ens
})

# jaga urutan sesuai sample_submission
if "Record number" in sub.columns:
    submission_cv = sub[["Record number"]].merge(submission_cv, on="Record number", how="left")

submission_cv.to_csv("submission_cv_ensemble.csv", index=False)
submission_cv.to_csv(ENS_DIR / "submission_cv_ensemble.csv", index=False)

print("✅ submission_cv_ensemble.csv dibuat!")
print(submission_cv.head(10))


✅ submission_cv_ensemble.csv dibuat!
   Record number  Turbidity
0          54916   5.512695
1          54917   5.568242
2          54918   6.136874
3          54919   5.659361
4          54920   6.280953
5          54921   5.993691
6          54922   6.280746
7          54923   6.345777
8          54924   6.995998
9          54925   6.289674


# random forest

In [10]:
# =========================================================
# Blok 0 — Setup, Paths, dan Library (RandomForest only)
# =========================================================
import os, gc, math, json, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.utils.validation import check_is_fitted

# Simpan/Load model
try:
    import joblib
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "joblib"])
    import joblib

# ====== Paths (prioritas Kaggle sesuai permintaan) ======
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback (opsional, jika dijalankan lokal)
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
rng = np.random.RandomState(SEED)
np.random.seed(SEED)



# =========================================================
# Blok 1 — Load Data, Sort Timestamp, dan Cek Kolom
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

# Parse timestamp dan urutkan
for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

TARGET = "Turbidity"
IDCOL  = "Record number"

# Daftar fitur top korelasi (sesuai list yang kamu kirim)
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]

# Cek kolom wajib
must_train = TOP_CORR_RAW + [TARGET, "Timestamp", IDCOL]
must_test  = TOP_CORR_RAW + ["Timestamp", IDCOL]
missing_train = [c for c in must_train if c not in train.columns]
missing_test  = [c for c in must_test  if c not in test.columns]
if missing_train: print("Peringatan, kolom hilang di train:", missing_train)
if missing_test:  print("Peringatan, kolom hilang di test :", missing_test)



# =========================================================
# Blok 2 — Gabung Train+Test & Estimasi Resolusi Waktu
# =========================================================
train["__is_train__"] = 1
test["__is_train__"]  = 0
full = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt) == 0:
        return 1
    med = np.median(dt)
    if not np.isfinite(med) or med <= 0:
        return 1
    return int(max(1, round(3600.0 / med)))

steps_per_hour = estimate_steps_per_hour(full)
steps_1h  = max(1, steps_per_hour)
steps_3h  = max(1, 3*steps_per_hour)
steps_6h  = max(1, 6*steps_per_hour)
steps_24h = max(1, 24*steps_per_hour)

print(f"Estimasi steps per hour: {steps_per_hour} | windows: 1h={steps_1h}, 3h={steps_3h}, 6h={steps_6h}, 24h={steps_24h}")



# =========================================================
# Blok 3 — Imputasi Time-Aware + Fitur Lag/Rolling (Top Korelasi)
# =========================================================
def timewise_impute(df, cols, steps_6h=12):
    out = df.copy()
    out[cols] = out[cols].ffill().bfill()             # ffill & bfill
    for c in cols:
        out[c] = out[c].fillna(out[c].rolling(window=steps_6h, min_periods=1).median())  # rolling median (ke belakang)
        out[c] = out[c].fillna(out[c].median())       # fallback median global
    return out

def add_lag_roll(df, cols, lags, rolls):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
        for w in rolls:
            out[f"{c}_roll{w}_mean"] = out[c].shift(1).rolling(window=w, min_periods=1).mean()
            out[f"{c}_roll{w}_std"]  = out[c].shift(1).rolling(window=w, min_periods=1).std()
            out[f"{c}_roll{w}_min"]  = out[c].shift(1).rolling(window=w, min_periods=1).min()
            out[f"{c}_roll{w}_max"]  = out[c].shift(1).rolling(window=w, min_periods=1).max()
    return out

# Sanitasi sederhana
full["pH"] = full["pH"].clip(lower=0, upper=14)

# Imputasi hanya fitur top-korelasi
full = timewise_impute(full, TOP_CORR_RAW, steps_6h=steps_6h)

# Tambah lag/rolling (anti-leakage pakai shift(1) untuk rolling)
LAGS  = [1, steps_1h, steps_3h]                 # 1-step, ~1 jam, ~3 jam
ROLLS = [steps_1h, steps_6h, steps_24h]         # ~1h, ~6h, ~24h
full = add_lag_roll(full, TOP_CORR_RAW, LAGS, ROLLS)

# Susun daftar fitur akhir
lagroll_feats = [c for c in full.columns if any(s in c for s in ["_lag", "_roll"])]
FEATURES = TOP_CORR_RAW + lagroll_feats
print(f"Jumlah fitur dipakai: {len(FEATURES)}")




# =========================================================
# Blok 4 — Split Train/Test (drop NaN awal), Matrix X/y
# =========================================================
train_fe = full[full["__is_train__"]==1].copy()
test_fe  = full[full["__is_train__"]==0].copy()
train_fe[TARGET] = train[TARGET].values  # align target

# Drop baris train yang masih NaN (efek lag/rolling di awal)
train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

train_X = train_fe[FEATURES].copy()
y       = train_fe[TARGET].values
test_X  = test_fe[FEATURES].copy()

# Isi NaN sisa di test dengan median train (aman untuk inference)
test_X = test_X.fillna(train_X.median())

print("train_X shape:", train_X.shape, "| test_X shape:", test_X.shape)




# =========================================================
# Blok 5 — TimeSeriesSplit CV (OOF) dengan RandomForest
# =========================================================
# Catatan: RF tidak punya early stopping.
# Kita pakai CV OOF untuk estimasi MSE & juga simpan prediksi test per-fold (CV-ensemble).

n_splits = 5
# Buffer 'gap' untuk memutus ketergantungan rolling besar
gap = max(1, steps_1h)  # bisa ditingkatkan, misal steps_6h atau steps_24h jika leakage terasa
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

rf_params = dict(
    n_estimators=800,
    max_depth=None,             # bisa dikunci (mis. 18-28) jika overfit
    max_features="sqrt",
    min_samples_split=4,
    min_samples_leaf=3,
    bootstrap=True,
    n_jobs=-1,
    random_state=SEED
)

oof_pred = np.zeros(len(train_fe))
test_fold_preds = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    rf = RandomForestRegressor(**rf_params)
    rf.fit(X_tr, y_tr)

    pred_va = rf.predict(X_va)
    oof_pred[va_idx] = np.clip(pred_va, 0, None)  # NTU tidak negatif

    # Simpan prediksi test per fold (untuk ensemble average)
    pred_te = rf.predict(test_X)
    pred_te = np.clip(pred_te, 0, None)
    test_fold_preds.append(pred_te)

    mse_fold = mean_squared_error(y[va_idx], oof_pred[va_idx])
    print(f"Fold {fold} — MSE={mse_fold:.6f}")

cv_mse = mean_squared_error(y, oof_pred)
print(f"\nCV MSE (OOF, RF) = {cv_mse:.6f}")



# =========================================================
# Blok 6 — Train Full RF & Prediksi Test (+ CV-Ensemble)
# =========================================================
# 1) Full-model: latih ulang di seluruh data
rf_full = RandomForestRegressor(**rf_params)
rf_full.fit(train_X, y)

pred_test_full = rf_full.predict(test_X)
pred_test_full = np.clip(pred_test_full, 0, None)

# 2) CV-ensemble: rata-rata prediksi test dari semua model fold
if len(test_fold_preds) > 0:
    pred_test_cv_ens = np.mean(np.vstack(test_fold_preds), axis=0)
    pred_test_cv_ens = np.clip(pred_test_cv_ens, 0, None)
else:
    pred_test_cv_ens = pred_test_full.copy()

print("Pred test (full)  mean/std:", float(np.mean(pred_test_full)), float(np.std(pred_test_full)))
print("Pred test (cvens) mean/std:", float(np.mean(pred_test_cv_ens)), float(np.std(pred_test_cv_ens)))





# =========================================================
# Blok 7 — Buat submission.csv (urut sesuai sample)
# =========================================================
# Default: pakai CV-ensemble (lebih stabil). Kalau mau pakai full model, ganti variabel di bawah.
FINAL_PRED = pred_test_cv_ens  # atau: pred_test_full

submission = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": FINAL_PRED
})

# Jaga urutan sesuai sample_submission
if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))





# =========================================================
# Blok 8 — Simpan Artefak (model, fitur, OOF, FI, metadata)
# =========================================================
OUT_DIR = Path("./outputs_rf")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Simpan kedua versi submission (full & ensemble)
submission.to_csv(OUT_DIR / "submission.csv", index=False)
pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": pred_test_full
}).to_csv(OUT_DIR / "submission_full_model.csv", index=False)

if len(test_fold_preds) > 0:
    pd.DataFrame({
        "Record number": test_fe[IDCOL].values,
        "Turbidity": pred_test_cv_ens
    }).to_csv(OUT_DIR / "submission_cv_ensemble.csv", index=False)

# Simpan model full
joblib.dump(rf_full, OUT_DIR / "rf_full.pkl")

# Simpan fitur (urutan penting untuk inference)
pd.Series(train_X.columns).to_csv(OUT_DIR / "features.txt", index=False, header=False)

# Simpan OOF pred
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_pred[:len(oof_df)]
oof_df.to_csv(OUT_DIR / "oof_predictions.csv", index=False)

# Feature importance (RF Gini importance)
try:
    fi = pd.DataFrame({
        "feature": train_X.columns,
        "importance": rf_full.feature_importances_
    }).sort_values("importance", ascending=False)
    fi.to_csv(OUT_DIR / "feature_importance.csv", index=False)
    print("Top 20 FI:")
    print(fi.head(20))
except Exception as e:
    print("Gagal mengambil feature_importances_:", e)

# Metadata
meta = {
    "cv_mse": float(cv_mse),
    "n_splits": int(n_splits),
    "gap": int(gap),
    "rf_params": rf_params,
    "steps_per_hour": int(steps_per_hour),
    "n_features": int(len(train_X.columns))
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Artefak disimpan di:", OUT_DIR.resolve())
print("📄 Files:", [p.name for p in OUT_DIR.iterdir()])


Estimasi steps per hour: 6 | windows: 1h=6, 3h=18, 6h=36, 24h=144
Jumlah fitur dipakai: 128
train_X shape: (47364, 128) | test_X shape: (14610, 128)
Fold 1 — MSE=5.247021
Fold 2 — MSE=38.993243
Fold 3 — MSE=118.627333
Fold 4 — MSE=19.076475
Fold 5 — MSE=51.116512

CV MSE (OOF, RF) = 39.705318
Pred test (full)  mean/std: 6.666789935990432 1.3255411832094708
Pred test (cvens) mean/std: 6.555483457049812 1.882542586905388
✅ submission.csv dibuat!
   Record number  Turbidity
0          54916   5.330515
1          54917   5.568005
2          54918   6.102316
3          54919   6.196658
4          54920   5.883959
5          54921   5.911880
6          54922   6.084784
7          54923   6.447823
8          54924   7.104583
9          54925   7.195050
Top 20 FI:
                                         feature  importance
34              Average Water Speed_roll144_mean    0.038460
126                      Chlorophyll_roll144_min    0.025130
82                                pH_roll144_max  

# XGBOST

In [11]:
# =========================================================
# Blok 0 — Setup, Paths, dan Library (XGBoost only)
# =========================================================
import os, gc, math, json, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Install XGBoost jika belum tersedia
try:
    import xgboost as xgb
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "xgboost"])
    import xgboost as xgb

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# Simpan/Load model
try:
    import joblib
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "joblib"])
    import joblib

# ====== Paths (prioritas Kaggle sesuai permintaan) ======
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback (opsional, jika ingin jalankan lokal)
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
np.random.seed(SEED)

# =========================================================
# Blok 1 — Load Data, Sort Timestamp, dan Cek Kolom
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

# Parse timestamp dan urutkan
for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

TARGET = "Turbidity"
IDCOL  = "Record number"

# Daftar fitur top korelasi (sesuai list yang kamu kirim)
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]

# Cek kolom wajib
must_train = TOP_CORR_RAW + [TARGET, "Timestamp", IDCOL]
must_test  = TOP_CORR_RAW + ["Timestamp", IDCOL]
missing_train = [c for c in must_train if c not in train.columns]
missing_test  = [c for c in must_test  if c not in test.columns]
if missing_train: print("Peringatan, kolom hilang di train:", missing_train)
if missing_test:  print("Peringatan, kolom hilang di test :", missing_test)


# =========================================================
# Blok 2 — Gabung Train+Test & Estimasi Resolusi Waktu
# =========================================================
train["__is_train__"] = 1
test["__is_train__"]  = 0
full = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt) == 0:
        return 1
    med = np.median(dt)
    if not np.isfinite(med) or med <= 0:
        return 1
    return int(max(1, round(3600.0 / med)))

steps_per_hour = estimate_steps_per_hour(full)
steps_1h  = max(1, steps_per_hour)
steps_3h  = max(1, 3*steps_per_hour)
steps_6h  = max(1, 6*steps_per_hour)
steps_24h = max(1, 24*steps_per_hour)

print(f"Estimasi steps per hour: {steps_per_hour} | windows: 1h={steps_1h}, 3h={steps_3h}, 6h={steps_6h}, 24h={steps_24h}")


# =========================================================
# Blok 3 — Imputasi Time-Aware + Fitur Lag/Rolling (Top Korelasi)
# =========================================================
def timewise_impute(df, cols, steps_6h=12):
    out = df.copy()
    out[cols] = out[cols].ffill().bfill()  # ffill & bfill
    for c in cols:
        out[c] = out[c].fillna(out[c].rolling(window=steps_6h, min_periods=1).median())  # rolling median (ke belakang)
        out[c] = out[c].fillna(out[c].median())  # fallback median global
    return out

def add_lag_roll(df, cols, lags, rolls):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
        for w in rolls:
            out[f"{c}_roll{w}_mean"] = out[c].shift(1).rolling(window=w, min_periods=1).mean()
            out[f"{c}_roll{w}_std"]  = out[c].shift(1).rolling(window=w, min_periods=1).std()
            out[f"{c}_roll{w}_min"]  = out[c].shift(1).rolling(window=w, min_periods=1).min()
            out[f"{c}_roll{w}_max"]  = out[c].shift(1).rolling(window=w, min_periods=1).max()
    return out

# Sanitasi sederhana
full["pH"] = full["pH"].clip(lower=0, upper=14)

# Imputasi hanya fitur top-korelasi
full = timewise_impute(full, TOP_CORR_RAW, steps_6h=steps_6h)

# Tambah lag/rolling (anti-leakage pakai shift(1) untuk rolling)
LAGS  = [1, steps_1h, steps_3h]                 # 1-step, ~1 jam, ~3 jam
ROLLS = [steps_1h, steps_6h, steps_24h]         # ~1h, ~6h, ~24h
full = add_lag_roll(full, TOP_CORR_RAW, LAGS, ROLLS)

# Susun daftar fitur akhir
lagroll_feats = [c for c in full.columns if any(s in c for s in ["_lag", "_roll"])]
FEATURES = TOP_CORR_RAW + lagroll_feats
print(f"Jumlah fitur dipakai: {len(FEATURES)}")

# =========================================================
# Blok 4 — Split Train/Test (drop NaN awal), Matrix X/y
# =========================================================
train_fe = full[full["__is_train__"]==1].copy()
test_fe  = full[full["__is_train__"]==0].copy()
train_fe[TARGET] = train[TARGET].values  # align target

# Drop baris train yang masih NaN (efek lag/rolling di awal)
train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

train_X = train_fe[FEATURES].copy()
y       = train_fe[TARGET].values
test_X  = test_fe[FEATURES].copy()

# Isi NaN sisa di test dengan median train (aman untuk inference)
test_X = test_X.fillna(train_X.median())

print("train_X shape:", train_X.shape, "| test_X shape:", test_X.shape)



# =========================================================
# Blok 5 — TimeSeriesSplit CV (XGBoost, Early Stopping, OOF)
# =========================================================
from xgboost import XGBRegressor

n_splits = 5
# Buffer 'gap' untuk memutus ketergantungan rolling besar
gap = max(1, steps_1h)  # bisa ditingkatkan ke steps_6h/steps_24h jika leakage terasa
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

xgb_params = dict(
    objective="reg:squarederror",  # MSE objective
    learning_rate=0.05,
    max_depth=8,
    min_child_weight=4,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    n_estimators=5000,             # besar, dipangkas oleh early_stopping
    random_state=SEED,
    tree_method="hist",            # aman & cepat di CPU; ganti "gpu_hist" jika yakin GPU tersedia
)

oof_pred = np.zeros(len(train_fe))
test_fold_preds = []
best_iters = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    model = XGBRegressor(**xgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="rmse",
        verbose=False,
        early_stopping_rounds=300
    )

    best_it = model.best_iteration if hasattr(model, "best_iteration") else model.n_estimators
    best_iters.append(int(best_it))

    pred_va = model.predict(X_va, iteration_range=(0, best_it))
    oof_pred[va_idx] = np.clip(pred_va, 0, None)  # NTU tidak negatif

    # Simpan prediksi test per fold (untuk ensemble)
    pred_te = model.predict(test_X, iteration_range=(0, best_it))
    pred_te = np.clip(pred_te, 0, None)
    test_fold_preds.append(pred_te)

    fold_mse = mean_squared_error(y[va_idx], oof_pred[va_idx])
    print(f"Fold {fold} — best_iter={best_it}, MSE={fold_mse:.6f}")

cv_mse = mean_squared_error(y, oof_pred)
best_iter_final = int(np.median(best_iters)) if len(best_iters)>0 else xgb_params["n_estimators"]
print(f"\nCV MSE (OOF, XGB) = {cv_mse:.6f} | median best_iter = {best_iter_final}")


# =========================================================
# Blok 6 — Train Full Model & Prediksi Test (+ CV-Ensemble)
# =========================================================
# 1) Full-model: latih ulang di seluruh data dengan n_estimators = median best_iter
xgb_full = XGBRegressor(**{**xgb_params, "n_estimators": best_iter_final})
xgb_full.fit(train_X, y, verbose=False)

pred_test_full = xgb_full.predict(test_X)
pred_test_full = np.clip(pred_test_full, 0, None)

# 2) CV-ensemble: rata-rata prediksi test dari semua model fold
if len(test_fold_preds) > 0:
    pred_test_cv_ens = np.mean(np.vstack(test_fold_preds), axis=0)
    pred_test_cv_ens = np.clip(pred_test_cv_ens, 0, None)
else:
    pred_test_cv_ens = pred_test_full.copy()

print("Pred test (full)  mean/std:", float(np.mean(pred_test_full)), float(np.std(pred_test_full)))
print("Pred test (cvens) mean/std:", float(np.mean(pred_test_cv_ens)), float(np.std(pred_test_cv_ens)))


# =========================================================
# Blok 7 — Buat submission.csv (urut sesuai sample)
# =========================================================
# Default: pakai CV-ensemble (umumnya lebih stabil). Kalau mau pakai full model, ganti variabel di bawah.
FINAL_PRED = pred_test_cv_ens  # atau: pred_test_full

submission = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": FINAL_PRED
})

# Jaga urutan sesuai sample_submission
if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))

# =========================================================
# Blok 8 — Simpan Artefak (model, fitur, OOF, FI, metadata)
# =========================================================
OUT_DIR = Path("./outputs_xgb")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Simpan submission (utama & alternatif)
submission.to_csv(OUT_DIR / "submission.csv", index=False)
pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": pred_test_full
}).to_csv(OUT_DIR / "submission_full_model.csv", index=False)

if len(test_fold_preds) > 0:
    pd.DataFrame({
        "Record number": test_fe[IDCOL].values,
        "Turbidity": pred_test_cv_ens
    }).to_csv(OUT_DIR / "submission_cv_ensemble.csv", index=False)

# Simpan model full
joblib.dump(xgb_full, OUT_DIR / "xgb_full.pkl")
# Simpan booster model JSON (portable ke XGBoost native)
xgb_full.get_booster().save_model(str(OUT_DIR / "xgb_full.json"))

# Simpan daftar fitur (urutan penting untuk inference)
pd.Series(train_X.columns).to_csv(OUT_DIR / "features.txt", index=False, header=False)

# Simpan OOF pred
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_pred[:len(oof_df)]
oof_df.to_csv(OUT_DIR / "oof_predictions.csv", index=False)

# Feature importance (gain)
try:
    booster = xgb_full.get_booster()
    gain_map = booster.get_score(importance_type="gain")  # dict: {feature_name: gain}
    # Pastikan nama fitur konsisten dengan kolom: XGBoost memetakan secara otomatis ke f0,f1... jika tidak pakai DMatrix
    # Untuk kestabilan, gunakan feature_importances_ dari wrapper sebagai fallback
    fi_gain = pd.DataFrame({
        "feature": list(train_X.columns),
        "importance": xgb_full.feature_importances_
    }).sort_values("importance", ascending=False)
    fi_gain.to_csv(OUT_DIR / "feature_importance.csv", index=False)
    print("Top 20 FI (wrapper-based):")
    print(fi_gain.head(20))
except Exception as e:
    print("Gagal mengambil feature importance:", e)

# Metadata
meta = {
    "cv_mse": float(cv_mse),
    "n_splits": int(n_splits),
    "gap": int(gap),
    "xgb_params": {**xgb_params, "n_estimators": int(best_iter_final)},
    "steps_per_hour": int(steps_per_hour),
    "n_features": int(len(train_X.columns))
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Artefak disimpan di:", OUT_DIR.resolve())
print("📄 Files:", [p.name for p in OUT_DIR.iterdir()])


# =========================================================
# Blok 9 — (Opsional) Reload Model & Cek Prediksi Cepat
# =========================================================
reloaded = joblib.load(OUT_DIR / "xgb_full.pkl")
test_pred_reload = np.clip(reloaded.predict(test_X), 0, None)
assert len(test_pred_reload) == len(test_fe), "Mismatch panjang prediksi vs test!"
print("Reload OK. Contoh 5 pred:", np.round(test_pred_reload[:5], 4))


Estimasi steps per hour: 6 | windows: 1h=6, 3h=18, 6h=36, 24h=144
Jumlah fitur dipakai: 128
train_X shape: (47364, 128) | test_X shape: (14610, 128)
Fold 1 — best_iter=1133, MSE=4.529689
Fold 2 — best_iter=1273, MSE=37.132218
Fold 3 — best_iter=0, MSE=13.490963
Fold 4 — best_iter=0, MSE=8.400354
Fold 5 — best_iter=15, MSE=47.766616

CV MSE (OOF, XGB) = 19.415194 | median best_iter = 15
Pred test (full)  mean/std: 5.488552570343018 1.47579026222229
Pred test (cvens) mean/std: 5.610215663909912 1.0423319339752197
✅ submission.csv dibuat!
   Record number  Turbidity
0          54916   5.688222
1          54917   6.173228
2          54918   7.083472
3          54919   6.831615
4          54920   6.278066
5          54921   6.215439
6          54922   6.873591
7          54923   7.334956
8          54924   7.366265
9          54925   7.181188
Top 20 FI (wrapper-based):
                                         feature  importance
111                      Temperature_roll144_min    0.070469
6

In [ ]:
train_fe.info()

# XGB Update

In [12]:
# =========================================================
# Blok 0 — Setup, Paths, Library
# =========================================================
import os, gc, math, json, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# XGBoost
try:
    import xgboost as xgb
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "xgboost"])
    import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

try:
    import joblib
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "joblib"])
    import joblib

# Paths Kaggle
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback lokal
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
np.random.seed(SEED)

TARGET = "Turbidity"
IDCOL  = "Record number"

# Top-korelasi (dari daftar kamu)
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]



# =========================================================
# Blok 1 — Load & Sort
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

# Cek kolom wajib
need_train = TOP_CORR_RAW + [TARGET, "Timestamp", IDCOL]
need_test  = TOP_CORR_RAW + ["Timestamp", IDCOL]
mt = [c for c in need_train if c not in train.columns]
ms = [c for c in need_test  if c not in test.columns]
if mt: print("⚠️ Missing train cols:", mt)
if ms: print("⚠️ Missing test cols :", ms)



# =========================================================
# Blok 2 — Gabung & Estimasi Resolusi Waktu
# =========================================================
train["__is_train__"] = 1
test["__is_train__"]  = 0
full = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt)==0: return 1
    med = np.median(dt)
    if not np.isfinite(med) or med<=0: return 1
    return int(max(1, round(3600.0/med)))

steps_per_hour = estimate_steps_per_hour(full)
steps_1h  = max(1, steps_per_hour)
steps_6h  = max(1, 6*steps_per_hour)
steps_12h = max(1, 12*steps_per_hour)
steps_24h = max(1, 24*steps_per_hour)
steps_48h = max(1, 48*steps_per_hour)

print(f"steps/hour={steps_per_hour} | 1h={steps_1h} 6h={steps_6h} 12h={steps_12h} 24h={steps_24h} 48h={steps_48h}")





# =========================================================
# Blok 3 — Flag Missingness & Imputasi Time-aware (no ffill lintas gap)
# =========================================================
# Flag missingness sebelum imputasi
for c in TOP_CORR_RAW:
    full[c+"_was_na"] = full[c].isna().astype(int)

# Batas "gap besar" (contoh: >6 jam) — optional; di sini kita tidak pakai ffill,
# jadi tak perlu memutus secara eksplisit. Dibiarkan sebagai referensi:
full["gap_sec"] = full["Timestamp"].diff().dt.total_seconds().fillna(0)

def impute_timeaware_past_rolling(df, cols, win=12):
    # Imputasi pakai rolling median dari MASA LALU (shift(1)) → tidak menyeberangkan informasi
    out = df.copy()
    for c in cols:
        past_med = out[c].shift(1).rolling(window=win, min_periods=1).median()
        out[c] = out[c].fillna(past_med)
        out[c] = out[c].fillna(out[c].median())  # fallback global
    return out

# Sanitasi ringan
full["pH"] = full["pH"].clip(0, 14)

# Imputasi: gunakan window ~6 jam
full = impute_timeaware_past_rolling(full, TOP_CORR_RAW, win=steps_6h)



# =========================================================
# Blok 4 — Lag & Rolling Features (tanpa std)
# =========================================================
def add_lag_features(df, cols, lags):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
    return out

def add_roll_features(df, cols, windows):
    out = df.copy()
    for c in cols:
        for w in windows:
            base = out[c].shift(1).rolling(window=w, min_periods=1)  # masa lalu saja
            out[f"{c}_roll{w}_mean"] = base.mean()
            out[f"{c}_roll{w}_min"]  = base.min()
            out[f"{c}_roll{w}_max"]  = base.max()
    return out

LAGS  = [1, steps_1h, steps_6h]  # 1-step, ~1h, ~6h
ROLLS = [steps_1h, steps_6h, steps_12h, steps_24h, steps_48h]  # 1h,6h,12h,24h,48h

full = add_lag_features(full, TOP_CORR_RAW, LAGS)
full = add_roll_features(full, TOP_CORR_RAW, ROLLS)

# Kumpulkan fitur final
lag_feats   = [c for c in full.columns if any(s in c for s in ["_lag"])]
roll_feats  = [c for c in full.columns if any(s in c for s in ["_roll"]) and c.endswith(("mean","min","max"))]
miss_flags  = [c+"_was_na" for c in TOP_CORR_RAW]
FEATURES    = TOP_CORR_RAW + miss_flags + lag_feats + roll_feats

print(f"n_features={len(FEATURES)}")



# =========================================================
# Blok 5 — Split Train/Test, Log-Target, Matrix X/y
# =========================================================
train_fe = full[full["__is_train__"]==1].copy()
test_fe  = full[full["__is_train__"]==0].copy()

# Align target ke urutan train_fe
train_fe[TARGET] = train[TARGET].values

# Buang baris awal yang masih NaN (efek lag/rolling)
train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

train_X = train_fe[FEATURES].copy()
test_X  = test_fe[FEATURES].copy()

# Isi NaN sisa pada test dengan median train
test_X = test_X.fillna(train_X.median())

# Log1p target (stabilisasi)
USE_LOG_TARGET = True
if USE_LOG_TARGET:
    y_raw = train_fe[TARGET].values
    y = np.log1p(y_raw)
else:
    y = train_fe[TARGET].values

print("train_X:", train_X.shape, "test_X:", test_X.shape, "| USE_LOG_TARGET:", USE_LOG_TARGET)




# =========================================================
# Blok 6 — TimeSeriesSplit CV (gap = 24h) & Training XGB
# =========================================================
# Gap minimal = window terpanjang (24h). Bisa dinaikkan ke 2*24h untuk ekstra aman.
GAP_MULTIPLIER = 1
gap = steps_24h * GAP_MULTIPLIER

n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

# Param XGB yang lebih stabil
xgb_params = dict(
    objective="reg:squarederror",
    learning_rate=0.03,
    max_depth=8,
    min_child_weight=8,
    subsample=0.8,
    colsample_bytree=0.7,
    reg_lambda=3.0,
    reg_alpha=0.2,
    n_estimators=6000,
    random_state=SEED,
    tree_method="hist",   # ganti ke 'gpu_hist' bila GPU XGB tersedia
)

oof_pred_log = np.zeros(len(train_fe))
test_fold_preds = []
best_iters = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    model = XGBRegressor(**xgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="rmse",
        verbose=False,
        early_stopping_rounds=500
    )

    best_it = model.best_iteration if hasattr(model, "best_iteration") else None
    best_iters.append(int(best_it) if best_it is not None else int(xgb_params["n_estimators"]))

    # Prediksi valid di ruang log (OOF tetap log), lalu inverse buat MSE report
    if best_it is None:
        pred_va_log = model.predict(X_va)
    else:
        pred_va_log = model.predict(X_va, iteration_range=(0, best_it))

    oof_pred_log[va_idx] = pred_va_log

    # Prediksi test (log), lalu simpan (nanti kita inverse & ensemble)
    if best_it is None:
        pred_te_log = model.predict(test_X)
    else:
        pred_te_log = model.predict(test_X, iteration_range=(0, best_it))
    test_fold_preds.append(pred_te_log)

    # MSE OOF dihitung pada ruang asli (expm1)
    y_va_raw = np.expm1(y_va) if USE_LOG_TARGET else y_va
    pred_va_raw = np.expm1(pred_va_log) if USE_LOG_TARGET else pred_va_log
    pred_va_raw = np.clip(pred_va_raw, 0, None)
    mse_fold = mean_squared_error(y_va_raw, pred_va_raw)
    print(f"Fold {fold} — best_iter={best_it}, MSE(raw)={mse_fold:.6f}")

best_iter_final = int(np.median(best_iters)) if len(best_iters)>0 else int(xgb_params["n_estimators"])

# Hitung CV MSE (OOF) di ruang asli
if USE_LOG_TARGET:
    oof_pred_raw = np.expm1(oof_pred_log)
else:
    oof_pred_raw = oof_pred_log
oof_pred_raw = np.clip(oof_pred_raw, 0, None)

cv_mse = mean_squared_error(train_fe[TARGET].values, oof_pred_raw)
print(f"\nCV MSE (OOF, raw) = {cv_mse:.6f} | median best_iter = {best_iter_final}")



# =========================================================
# Blok 7 — Train Full Model & Prediksi Test (Full + CV-Ensemble)
# =========================================================
# Full-model di ruang log (jika USE_LOG_TARGET)
xgb_full = XGBRegressor(**{**xgb_params, "n_estimators": best_iter_final})
xgb_full.fit(train_X, y, verbose=False)

pred_test_full_log = xgb_full.predict(test_X)
pred_test_cvens_log = np.mean(np.vstack(test_fold_preds), axis=0) if len(test_fold_preds)>0 else pred_test_full_log

# Inverse ke ruang asli & clip non-negatif
pred_test_full  = np.expm1(pred_test_full_log)  if USE_LOG_TARGET else pred_test_full_log
pred_test_cvens = np.expm1(pred_test_cvens_log) if USE_LOG_TARGET else pred_test_cvens_log
pred_test_full  = np.clip(pred_test_full,  0, None)
pred_test_cvens = np.clip(pred_test_cvens, 0, None)

print("Pred test (full)  mean/std:", float(pred_test_full.mean()), float(pred_test_full.std()))
print("Pred test (cvens) mean/std:", float(pred_test_cvens.mean()), float(pred_test_cvens.std()))



# =========================================================
# Blok 8 — Submission (pakai CV-ensemble by default)
# =========================================================
FINAL_PRED = pred_test_cvens  # ganti ke pred_test_full jika ingin full-model

submission = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": FINAL_PRED
})

# Jaga urutan sesuai sample
if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))





# =========================================================
# Blok 9 — Artefak (model, fitur, OOF, FI, metadata)
# =========================================================
OUT_DIR = Path("./outputs_xgb_fixed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Simpan submission alternatif
submission.to_csv(OUT_DIR / "submission.csv", index=False)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": pred_test_full}).to_csv(
    OUT_DIR / "submission_full_model.csv", index=False
)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": pred_test_cvens}).to_csv(
    OUT_DIR / "submission_cv_ensemble.csv", index=False
)

# Simpan model & booster
joblib.dump(xgb_full, OUT_DIR / "xgb_full.pkl")
xgb_full.get_booster().save_model(str(OUT_DIR / "xgb_full.json"))

# Simpan daftar fitur
pd.Series(train_X.columns).to_csv(OUT_DIR / "features.txt", index=False, header=False)

# Simpan OOF (ruang asli)
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_pred_raw[:len(oof_df)]
oof_df.to_csv(OUT_DIR / "oof_predictions.csv", index=False)

# FI (wrapper-based)
fi_gain = pd.DataFrame({
    "feature": train_X.columns,
    "importance": xgb_full.feature_importances_
}).sort_values("importance", ascending=False)
fi_gain.to_csv(OUT_DIR / "feature_importance.csv", index=False)
print("Top 20 FI:")
print(fi_gain.head(20))

# Metadata
meta = {
    "cv_mse_raw": float(cv_mse),
    "n_splits": int(n_splits),
    "gap_steps": int(gap),
    "gap_hours": float(gap / steps_per_hour),
    "xgb_params": {**xgb_params, "n_estimators": int(best_iter_final)},
    "steps_per_hour": int(steps_per_hour),
    "n_features": int(len(train_X.columns)),
    "use_log_target": bool(USE_LOG_TARGET)
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Artefak disimpan di:", OUT_DIR.resolve())
print("📄 Files:", [p.name for p in OUT_DIR.iterdir()])





# =========================================================
# Blok 10 — (Opsional) Reload & Sanity Check
# =========================================================
reloaded = joblib.load(OUT_DIR / "xgb_full.pkl")
reload_pred = reloaded.predict(test_X)
reload_pred = np.expm1(reload_pred) if USE_LOG_TARGET else reload_pred
reload_pred = np.clip(reload_pred, 0, None)
print("Reload OK. 5 preds:", np.round(reload_pred[:5], 4))


steps/hour=6 | 1h=6 6h=36 12h=72 24h=144 48h=288
n_features=160
train_X: (47346, 160) test_X: (14610, 160) | USE_LOG_TARGET: True
Fold 1 — best_iter=1670, MSE(raw)=5.605664
Fold 2 — best_iter=2042, MSE(raw)=38.242518
Fold 3 — best_iter=10, MSE(raw)=13.649890
Fold 4 — best_iter=9, MSE(raw)=9.500141
Fold 5 — best_iter=256, MSE(raw)=42.655219

CV MSE (OOF, raw) = 19.136668 | median best_iter = 256
Pred test (full)  mean/std: 5.373887538909912 1.3802059888839722
Pred test (cvens) mean/std: 4.224207878112793 0.6610403656959534
✅ submission.csv dibuat!
   Record number  Turbidity
0          54916   3.834093
1          54917   3.826866
2          54918   3.824403
3          54919   3.828991
4          54920   3.852460
5          54921   3.958128
6          54922   3.822677
7          54923   3.820855
8          54924   3.880584
9          54925   4.476418
Top 20 FI:
                                        feature  importance
52            Specific Conductance_roll288_mean    0.125424
54      

# Catbost

In [ ]:
# =========================================================
# Blok 0 — Setup, Paths, dan Library (CatBoost only)
# =========================================================
import os, gc, math, json, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Install CatBoost jika belum ada
try:
    from catboost import CatBoostRegressor, Pool
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "catboost"])
    from catboost import CatBoostRegressor, Pool

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

try:
    import joblib
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "joblib"])
    import joblib

# ====== Paths (prioritas Kaggle) ======
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback lokal
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
np.random.seed(SEED)

TARGET = "Turbidity"
IDCOL  = "Record number"

# Top-korelasi dari kamu
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]


# =========================================================
# Blok 1 — Load, Parse Timestamp, Sort, Sanity Checks
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

need_train = TOP_CORR_RAW + [TARGET, "Timestamp", IDCOL]
need_test  = TOP_CORR_RAW + ["Timestamp", IDCOL]
mt = [c for c in need_train if c not in train.columns]
ms = [c for c in need_test  if c not in test.columns]
if mt: print("⚠️ Missing train cols:", mt)
if ms: print("⚠️ Missing test cols :", ms)



# =========================================================
# Blok 2 — Gabung & Estimasi Resolusi Waktu (steps/hour)
# =========================================================
train["__is_train__"] = 1
test["__is_train__"]  = 0
full = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt)==0: return 1
    med = np.median(dt)
    if not np.isfinite(med) or med<=0: return 1
    return int(max(1, round(3600.0/med)))

steps_per_hour = estimate_steps_per_hour(full)
steps_1h  = max(1, steps_per_hour)
steps_6h  = max(1, 6*steps_per_hour)
steps_12h = max(1, 12*steps_per_hour)
steps_24h = max(1, 24*steps_per_hour)
steps_48h = max(1, 48*steps_per_hour)

print(f"steps/hour={steps_per_hour} | 1h={steps_1h} 6h={steps_6h} 12h={steps_12h} 24h={steps_24h} 48h={steps_48h}")



# =========================================================
# Blok 3 — Flag Missingness & Imputasi Time-Aware
# =========================================================
# Flag missingness sebelum imputasi
for c in TOP_CORR_RAW:
    full[c+"_was_na"] = full[c].isna().astype(int)

# Sanitasi ringan
full["pH"] = full["pH"].clip(0, 14)

def impute_timeaware_past_rolling(df, cols, win):
    out = df.copy()
    for c in cols:
        past_med = out[c].shift(1).rolling(window=win, min_periods=1).median()
        out[c] = out[c].fillna(past_med)
        out[c] = out[c].fillna(out[c].median())  # fallback global
    return out

# Imputasi dengan window ~6 jam (masa lalu saja)
full = impute_timeaware_past_rolling(full, TOP_CORR_RAW, win=steps_6h)



# =========================================================
# Blok 4 — Lag & Rolling Features (tanpa std)
# =========================================================
def add_lag_features(df, cols, lags):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
    return out

def add_roll_features(df, cols, windows):
    out = df.copy()
    for c in cols:
        for w in windows:
            past = out[c].shift(1).rolling(window=w, min_periods=1)  # masa lalu saja
            out[f"{c}_roll{w}_mean"] = past.mean()
            out[f"{c}_roll{w}_min"]  = past.min()
            out[f"{c}_roll{w}_max"]  = past.max()
    return out

LAGS  = [1, steps_1h, steps_6h]                          # 1-step, 1h, 6h
ROLLS = [steps_1h, steps_6h, steps_12h, steps_24h, steps_48h]  # 1h,6h,12h,24h,48h

full = add_lag_features(full, TOP_CORR_RAW, LAGS)
full = add_roll_features(full, TOP_CORR_RAW, ROLLS)

lag_feats   = [c for c in full.columns if c.endswith(tuple([f"_lag{L}" for L in LAGS]))]
roll_feats  = [c for c in full.columns if any(s in c for s in ["_roll"]) and c.endswith(("mean","min","max"))]
miss_flags  = [c+"_was_na" for c in TOP_CORR_RAW]
FEATURES    = TOP_CORR_RAW + miss_flags + lag_feats + roll_feats

print(f"n_features={len(FEATURES)}")



# =========================================================
# Blok 5 — Split Train/Test, Log-Target, Matrix X/y
# =========================================================
train_fe = full[full["__is_train__"]==1].copy()
test_fe  = full[full["__is_train__"]==0].copy()
train_fe[TARGET] = train[TARGET].values  # align target

# Drop baris awal (lag/roll NaN)
train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

train_X = train_fe[FEATURES].copy()
test_X  = test_fe[FEATURES].copy()
# Isi NaN sisa test dengan median train
test_X = test_X.fillna(train_X.median())

USE_LOG_TARGET = True
y_raw = train_fe[TARGET].values
y = np.log1p(y_raw) if USE_LOG_TARGET else y_raw

print("train_X:", train_X.shape, "test_X:", test_X.shape, "| USE_LOG_TARGET:", USE_LOG_TARGET)



# =========================================================
# Blok 6 — TimeSeriesSplit CV (gap = 24h) & Training CatBoost
# =========================================================
# Gap minimal = window terpanjang (24h)
GAP_MULTIPLIER = 1
gap = steps_24h * GAP_MULTIPLIER

n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

# Param CatBoost yang stabil
cb_params = dict(
    loss_function="RMSE",        # bekerja di ruang log (jika USE_LOG_TARGET)
    depth=8,
    learning_rate=0.03,
    l2_leaf_reg=6.0,
    random_seed=SEED,
    iterations=6000,
    eval_metric="RMSE",
    od_type="Iter",
    od_wait=500,                 # early stopping patience
    random_strength=1.5,
    rsm=0.7,                     # feature sampling (analog colsample)
    subsample=0.8,               # stochastic gradient boosting
    verbose=False,
    allow_writing_files=False,
    # task_type="CPU",           # uncomment untuk force CPU; bisa "GPU" jika tersedia
)

oof_pred_log = np.zeros(len(train_fe))
test_fold_preds_log = []
best_iters = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    tr_pool = Pool(data=X_tr, label=y_tr)
    va_pool = Pool(data=X_va, label=y_va)

    model = CatBoostRegressor(**cb_params)
    model.fit(tr_pool, eval_set=va_pool)

    # CatBoost tidak expose best_iteration langsung; ambil dari model.get_best_iteration()
    best_it = getattr(model, "get_best_iteration", lambda: None)()
    best_iters.append(int(best_it) if best_it is not None else int(cb_params["iterations"]))

    pred_va_log = model.predict(va_pool)
    oof_pred_log[va_idx] = pred_va_log

    te_pool = Pool(data=test_X)
    pred_te_log = model.predict(te_pool)
    test_fold_preds_log.append(pred_te_log)

    # OOF MSE pada ruang asli
    y_va_raw = np.expm1(y_va) if USE_LOG_TARGET else y_va
    pred_va_raw = np.expm1(pred_va_log) if USE_LOG_TARGET else pred_va_log
    pred_va_raw = np.clip(pred_va_raw, 0, None)
    mse_fold = mean_squared_error(y_va_raw, pred_va_raw)
    print(f"Fold {fold} — best_iter={best_it}, MSE(raw)={mse_fold:.6f}")

best_iter_final = int(np.median(best_iters)) if len(best_iters)>0 else int(cb_params["iterations"])

# CV MSE (OOF) ruang asli
oof_pred_raw = np.expm1(oof_pred_log) if USE_LOG_TARGET else oof_pred_log
oof_pred_raw = np.clip(oof_pred_raw, 0, None)
cv_mse = mean_squared_error(train_fe[TARGET].values, oof_pred_raw)
print(f"\nCV MSE (OOF, raw) = {cv_mse:.6f} | median best_iter = {best_iter_final}")



# =========================================================
# Blok 7 — Train Full Model & Prediksi Test (Full + CV-Ensemble)
# =========================================================
full_pool = Pool(data=train_X, label=y)
cb_full = CatBoostRegressor(**{**cb_params, "iterations": best_iter_final})
cb_full.fit(full_pool)

test_pool = Pool(data=test_X)
pred_test_full_log  = cb_full.predict(test_pool)
pred_test_cvens_log = np.mean(np.vstack(test_fold_preds_log), axis=0) if len(test_fold_preds_log)>0 else pred_test_full_log

# Inverse ke ruang asli
pred_test_full  = np.expm1(pred_test_full_log)  if USE_LOG_TARGET else pred_test_full_log
pred_test_cvens = np.expm1(pred_test_cvens_log) if USE_LOG_TARGET else pred_test_cvens_log
pred_test_full  = np.clip(pred_test_full,  0, None)
pred_test_cvens = np.clip(pred_test_cvens, 0, None)

print("Pred test (full)  mean/std:", float(pred_test_full.mean()), float(pred_test_full.std()))
print("Pred test (cvens) mean/std:", float(pred_test_cvens.mean()), float(pred_test_cvens.std()))



# =========================================================
# Blok 8 — Buat submission.csv (default pakai CV-ensemble)
# =========================================================
FINAL_PRED = pred_test_cvens  # ganti ke pred_test_full jika ingin full-model

submission = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": FINAL_PRED
})

# Jaga urutan sesuai sample submission
if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))




# =========================================================
# Blok 9 — Simpan Artefak (model, fitur, OOF, FI, metadata)
# =========================================================
OUT_DIR = Path("./outputs_catboost")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Submission alternatif
submission.to_csv(OUT_DIR / "submission.csv", index=False)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": pred_test_full}).to_csv(
    OUT_DIR / "submission_full_model.csv", index=False
)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": pred_test_cvens}).to_csv(
    OUT_DIR / "submission_cv_ensemble.csv", index=False
)

# Simpan full model (format .cbm) & joblib dump
cb_full.save_model(str(OUT_DIR / "catboost_full.cbm"))
joblib.dump(cb_full, OUT_DIR / "catboost_full.pkl")

# Daftar fitur (urutan penting saat inference)
pd.Series(train_X.columns).to_csv(OUT_DIR / "features.txt", index=False, header=False)

# Simpan OOF (ruang asli)
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_pred_raw[:len(oof_df)]
oof_df.to_csv(OUT_DIR / "oof_predictions.csv", index=False)

# Feature importance (PredictionValuesChange)
fi_vals = cb_full.get_feature_importance(full_pool, type="PredictionValuesChange")
fi = pd.DataFrame({"feature": train_X.columns, "importance": fi_vals}).sort_values("importance", ascending=False)
fi.to_csv(OUT_DIR / "feature_importance.csv", index=False)
print("Top 20 FI:")
print(fi.head(20))

# Metadata
meta = {
    "cv_mse_raw": float(cv_mse),
    "n_splits": int(n_splits),
    "gap_steps": int(gap),
    "gap_hours": float(gap / steps_per_hour),
    "cb_params": {**cb_params, "iterations": int(best_iter_final)},
    "steps_per_hour": int(steps_per_hour),
    "n_features": int(len(train_X.columns)),
    "use_log_target": bool(USE_LOG_TARGET)
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Artefak disimpan di:", OUT_DIR.resolve())
print("📄 Files:", [p.name for p in OUT_DIR.iterdir()])



# =========================================================
# Blok 10 — (Opsional) Reload & Sanity Check
# =========================================================
# Load .cbm kembali (via CatBoostRegressor.load_model)
cb_reload = CatBoostRegressor()
cb_reload.load_model(str(OUT_DIR / "catboost_full.cbm"))

reload_pred_log = cb_reload.predict(Pool(data=test_X))
reload_pred = np.expm1(reload_pred_log) if USE_LOG_TARGET else reload_pred_log
reload_pred = np.clip(reload_pred, 0, None)

print("Reload OK. 5 preds:", np.round(reload_pred[:5], 4))





steps/hour=6 | 1h=6 6h=36 12h=72 24h=144 48h=288
n_features=160
train_X: (47346, 160) test_X: (14610, 160) | USE_LOG_TARGET: True
Fold 1 — best_iter=2315, MSE(raw)=6.277053


# LGBM

In [14]:
# =========================================================
# Blok 0 — Setup, Paths, Library
# =========================================================
import os, gc, math, json, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# LightGBM
try:
    import lightgbm as lgb
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "lightgbm"])
    import lightgbm as lgb

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

try:
    import joblib
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "joblib"])
    import joblib

# ====== Paths (Kaggle) ======
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback lokal
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
np.random.seed(SEED)

TARGET = "Turbidity"
IDCOL  = "Record number"

# Fitur top-korelasi yang dipakai
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]


# =========================================================
# Blok 1 — Load, Parse Timestamp, Sort, Sanity Checks
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

need_train = TOP_CORR_RAW + [TARGET, "Timestamp", IDCOL]
need_test  = TOP_CORR_RAW + ["Timestamp", IDCOL]
mt = [c for c in need_train if c not in train.columns]
ms = [c for c in need_test  if c not in test.columns]
if mt: print("⚠️ Missing train cols:", mt)
if ms: print("⚠️ Missing test cols :", ms)



# =========================================================
# Blok 2 — Estimasi Resolusi Waktu (steps/hour)
# =========================================================
def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt)==0: return 1
    med = np.median(dt)
    if not np.isfinite(med) or med<=0: return 1
    return int(max(1, round(3600.0/med)))

# Pakai TRAIN untuk estimasi resolusi utama
steps_per_hour = estimate_steps_per_hour(train)
steps_1h  = max(1, steps_per_hour)
steps_6h  = max(1, 6*steps_per_hour)
steps_12h = max(1, 12*steps_per_hour)
steps_24h = max(1, 24*steps_per_hour)
steps_48h = max(1, 48*steps_per_hour)

print(f"steps/hour={steps_per_hour} | 1h={steps_1h} 6h={steps_6h} 12h={steps_12h} 24h={steps_24h} 48h={steps_48h}")



# =========================================================
# Blok 3 — Flag Missingness (FULL RAW, sebelum imputasi)
# =========================================================
# Kita hanya buat flag di sini. Imputasi akan dilakukan per-segmen (train & test) terpisah.
train["_seg"] = "train"
test["_seg"]  = "test"
full_raw = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

for c in TOP_CORR_RAW:
    full_raw[c+"_was_na"] = full_raw[c].isna().astype(int)

# Simpan flags per segmen
train_flags = full_raw.loc[full_raw["_seg"]=="train", [c+"_was_na" for c in TOP_CORR_RAW]].reset_index(drop=True)
test_flags  = full_raw.loc[full_raw["_seg"]=="test",  [c+"_was_na" for c in TOP_CORR_RAW]].reset_index(drop=True)


# =========================================================
# Blok 4 — Imputasi TRAIN (TRAIN-only)
# =========================================================
# pH sanitasi
train["pH"] = train["pH"].clip(0, 14)

# Fallback median TRAIN-ONLY
train_medians = {c: train[c].median(skipna=True) for c in TOP_CORR_RAW}

def impute_train_timeaware(df, cols, roll_win, ffill_limit, med_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        # 1) Past-rolling median (no future)
        past_med = s.shift(1).rolling(window=roll_win, min_periods=1).median()
        s = s.where(~s.isna(), past_med)
        # 2) Bounded forward-fill (≤ 6 jam)
        s = s.ffill(limit=ffill_limit)
        # 3) Fallback median TRAIN-ONLY
        s = s.fillna(med_map.get(c, np.nanmedian(out[c].values)))
        out[c] = s
    return out

train_imp = impute_train_timeaware(
    df=train,
    cols=TOP_CORR_RAW,
    roll_win=steps_6h,
    ffill_limit=steps_6h,
    med_map=train_medians
)

# Last-known (hasil imputasi) per kolom pada AKHIR TRAIN — dipakai untuk “seed” test
last_known_from_train = {}
for c in TOP_CORR_RAW:
    last_val = train_imp[c].iloc[-1]
    if pd.isna(last_val):
        last_val = train_medians[c]
    last_known_from_train[c] = last_val



# =========================================================
# Blok 5 — Imputasi TEST (tanpa statistik dari test)
# =========================================================
# Catatan:
# - Tidak menghitung rolling median dari test.
# - Jika awal test kosong, seed dengan last-known dari train (maks ffill 6 jam).
# - Sisanya ffill (≤ 6 jam) + fallback median TRAIN-ONLY.

test["pH"] = test["pH"].clip(0, 14)

def impute_test_seeded(df, cols, ffill_limit, med_map, seed_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()

        # Seed awal dengan last-known dari train jika nilai awal NA
        if len(s)>0 and pd.isna(s.iloc[0]) and pd.notna(seed_map.get(c, np.nan)):
            s.iloc[0] = seed_map[c]

        # Bounded forward-fill di dalam test (≤ 6 jam)
        s = s.ffill(limit=ffill_limit)

        # Fallback median TRAIN-ONLY
        s = s.fillna(med_map.get(c, np.nanmedian(out[c].values)))

        out[c] = s
    return out

test_imp = impute_test_seeded(
    df=test,
    cols=TOP_CORR_RAW,
    ffill_limit=steps_6h,
    med_map=train_medians,
    seed_map=last_known_from_train
)



# =========================================================
# Blok 6 — Gabungkan hasil imputasi, lalu Buat Fitur Lag/Rolling (anti-leakage)
# =========================================================
# Rekatkan kembali hasil imputasi (untuk fitur turunan)
train_imp["_seg"] = "train"
test_imp["_seg"]  = "test"
full_imp = pd.concat([train_imp, test_imp], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

# Tambah flags yang sudah dibuat dari full_raw (agar baris/urutan cocok)
flag_cols = [c+"_was_na" for c in TOP_CORR_RAW]
full_flags = pd.concat([train_flags, test_flags], axis=0, ignore_index=True)
for i, col in enumerate(flag_cols):
    full_imp[col] = full_flags[col].values

def add_lag_features(df, cols, lags):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
    return out

def add_roll_features(df, cols, windows):
    out = df.copy()
    for c in cols:
        for w in windows:
            past = out[c].shift(1).rolling(window=w, min_periods=1)  # masa lalu saja
            out[f"{c}_roll{w}_mean"] = past.mean()
            out[f"{c}_roll{w}_min"]  = past.min()
            out[f"{c}_roll{w}_max"]  = past.max()
    return out

LAGS  = [1, steps_1h, steps_6h]                          # 1-step, 1h, 6h
ROLLS = [steps_1h, steps_6h, steps_12h, steps_24h, steps_48h]  # 1h,6h,12h,24h,48h

full_imp = add_lag_features(full_imp, TOP_CORR_RAW, LAGS)
full_imp = add_roll_features(full_imp, TOP_CORR_RAW, ROLLS)

lag_feats   = [c for c in full_imp.columns if "_lag" in c]
roll_feats  = [c for c in full_imp.columns if "_roll" in c and c.endswith(("mean","min","max"))]
miss_flags  = [c+"_was_na" for c in TOP_CORR_RAW]
FEATURES    = TOP_CORR_RAW + miss_flags + lag_feats + roll_feats

print(f"n_features={len(FEATURES)}")



# =========================================================
# Blok 7 — Split Train/Test Matrices (drop NaN awal lag/roll)
# =========================================================
train_fe = full_imp[full_imp["_seg"]=="train"].copy()
test_fe  = full_imp[full_imp["_seg"]=="test"].copy()

# Align target ke urutan train_fe
train_fe[TARGET] = train[TARGET].values

# Drop baris awal train yang masih NaN (efek lag/rolling)
train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

train_X = train_fe[FEATURES].copy()
y       = train_fe[TARGET].values
test_X  = test_fe[FEATURES].copy()

# Isi NaN sisa test dengan median TRAIN (aman)
test_X = test_X.fillna(train_X.median())

print("train_X:", train_X.shape, "test_X:", test_X.shape)




# =========================================================
# Blok 8 — TimeSeriesSplit CV (gap = 24h) & LGBM Training
# =========================================================
# Gap minimal = window terpanjang (24h)
GAP_MULTIPLIER = 1
gap = steps_24h * GAP_MULTIPLIER

n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

lgb_params = dict(
    objective="l2",            # MSE
    learning_rate=0.05,
    num_leaves=64,
    feature_fraction=0.9,
    bagging_fraction=0.8,
    bagging_freq=1,
    min_data_in_leaf=60,
    lambda_l2=2.0,             # regulasi halus
    n_estimators=6000,         # besar; early_stopping memangkas
    random_state=SEED,
    verbose=-1
)

oof_pred = np.zeros(len(train_fe))
test_fold_preds = []
best_iters = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=500, verbose=False)]
    )

    best_it = model.best_iteration_
    best_iters.append(int(best_it) if best_it is not None else int(lgb_params["n_estimators"]))

    pred_va = model.predict(X_va, num_iteration=best_it)
    oof_pred[va_idx] = np.clip(pred_va, 0, None)

    pred_te = model.predict(test_X, num_iteration=best_it)
    pred_te = np.clip(pred_te, 0, None)
    test_fold_preds.append(pred_te)

    mse_fold = mean_squared_error(y[va_idx], oof_pred[va_idx])
    print(f"Fold {fold} — best_iter={best_it}, MSE={mse_fold:.6f}")

best_iter_final = int(np.median(best_iters)) if len(best_iters)>0 else int(lgb_params["n_estimators"])
cv_mse = mean_squared_error(y, oof_pred)
print(f"\nCV MSE (OOF, raw) = {cv_mse:.6f} | median best_iter = {best_iter_final}")



# =========================================================
# Blok 9 — Train Full Model & Prediksi Test (Full + CV-Ensemble)
# =========================================================
lgb_full = lgb.LGBMRegressor(**{**lgb_params, "n_estimators": best_iter_final})
lgb_full.fit(train_X, y)

pred_test_full  = np.clip(lgb_full.predict(test_X), 0, None)
pred_test_cvens = np.clip(np.mean(np.vstack(test_fold_preds), axis=0), 0, None) if len(test_fold_preds)>0 else pred_test_full

print("Pred test (full)  mean/std:", float(pred_test_full.mean()),  float(pred_test_full.std()))
print("Pred test (cvens) mean/std:", float(pred_test_cvens.mean()), float(pred_test_cvens.std()))



# =========================================================
# Blok 10 — Submission (default: CV-ensemble)
# =========================================================
FINAL_PRED = pred_test_cvens  # ganti ke pred_test_full jika ingin full-model

submission = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": FINAL_PRED
})

# Jaga urutan sesuai sample
if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))




# =========================================================
# Blok 11 — Artefak (model, fitur, OOF, FI, metadata)
# =========================================================
OUT_DIR = Path("./outputs_lgbm_trainonly_impute")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Submission alternatif
submission.to_csv(OUT_DIR / "submission.csv", index=False)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": pred_test_full}).to_csv(
    OUT_DIR / "submission_full_model.csv", index=False
)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": FINAL_PRED}).to_csv(
    OUT_DIR / "submission_cv_ensemble.csv", index=False
)

# Simpan model (booster + sklearn wrapper)
lgb_full.booster_.save_model(str(OUT_DIR / "lgbm_full.txt"))
joblib.dump(lgb_full, OUT_DIR / "lgbm_full.pkl")

# Fitur (urutan penting untuk inference)
pd.Series(train_X.columns).to_csv(OUT_DIR / "features.txt", index=False, header=False)

# OOF (ruang asli)
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_pred[:len(oof_df)]
oof_df.to_csv(OUT_DIR / "oof_predictions.csv", index=False)

# Feature importance (gain & split)
fi = pd.DataFrame({
    "feature": train_X.columns,
    "gain":   lgb_full.booster_.feature_importance(importance_type="gain"),
    "split":  lgb_full.booster_.feature_importance(importance_type="split"),
}).sort_values("gain", ascending=False)
fi.to_csv(OUT_DIR / "feature_importance.csv", index=False)
print("Top 20 FI:")
print(fi.head(20))

# Metadata
meta = {
    "cv_mse_raw": float(cv_mse),
    "n_splits": int(n_splits),
    "gap_steps": int(gap),
    "gap_hours": float(gap / steps_per_hour),
    "lgb_params": {**lgb_params, "n_estimators": int(best_iter_final)},
    "steps_per_hour": int(steps_per_hour),
    "n_features": int(len(train_X.columns)),
    "impute_policy": {
        "train": "past_rolling_median -> ffill<=6h -> fallback median TRAIN-only",
        "test":  "seed last-known(train) -> ffill<=6h -> fallback median TRAIN-only",
    }
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Artefak disimpan di:", OUT_DIR.resolve())
print("📄 Files:", [p.name for p in OUT_DIR.iterdir()])



steps/hour=6 | 1h=6 6h=36 12h=72 24h=144 48h=288
n_features=160
train_X: (47346, 160) test_X: (14610, 160)
Fold 1 — best_iter=2368, MSE=4.384105
Fold 2 — best_iter=3492, MSE=34.831326
Fold 3 — best_iter=2, MSE=12.900845
Fold 4 — best_iter=2, MSE=8.374097
Fold 5 — best_iter=174, MSE=38.176619

CV MSE (OOF, raw) = 17.305595 | median best_iter = 174
Pred test (full)  mean/std: 8.049893272593625 2.790662104118727
Pred test (cvens) mean/std: 5.83386998373311 0.7640241522932113
✅ submission.csv dibuat!
   Record number  Turbidity
0          54916   6.451931
1          54917   6.805037
2          54918   7.151798
3          54919   6.930721
4          54920   7.270678
5          54921   7.220288
6          54922   7.023513
7          54923   7.027749
8          54924   7.091690
9          54925   7.596410
Top 20 FI:
                                         feature           gain  split
158                      Chlorophyll_roll288_min  779707.415421    127
97   Dissolved Oxygen (%Saturation)_r

In [15]:
# =========================================================
# Blok 12 — Dokumentasi (Kontrol MSE & Catatan Rekayasa Fitur)
# =========================================================
import io

# --- 12.1 Tabel Kontrol MSE dari teks yang kamu berikan ---
controls_csv = """Area,Teknik Pengendalian,Keterangan
Penanganan Outlier,"Hapus, Transformasi Logaritma, atau Capping.","Kunci penting untuk MSE, karena outlier menghasilkan kesalahan besar yang sangat memengaruhi nilai MSE."
Penanganan Missing Values,"Imputasi (dengan rata-rata/median), Imputasi Deret Waktu (Interpolasi).","Data yang hilang harus ditangani agar tidak mengganggu pola dalam deret waktu."
Normalisasi/Standardisasi,"Min-Max Scaling atau Z-score Standardization.","Memastikan semua fitur berkontribusi secara proporsional, mencegah fitur dengan skala besar mendominasi proses pelatihan."
"""
controls_df = pd.read_csv(io.StringIO(controls_csv))
controls_path = OUT_DIR / "mse_controls.csv"
controls_df.to_csv(controls_path, index=False)

# --- 12.2 Catatan modeling (Markdown) dari poin-poin tambahan ---
notes_md = f"""# Catatan Modeling & MSE — LGBM Turbidity

## Kontrol MSE (ringkas)
Lihat **mse_controls.csv** untuk tabel pengendalian (outlier, missing values, normalisasi/standardisasi).

## Rekayasa Fitur (rangkuman)
- **Pembuatan Fitur Lag**: Untuk deret waktu, membuat fitur dari nilai historis **variabel independen** (Suhu/Temperature, DO, pH, dll.) pada langkah waktu sebelumnya (t-1, t-6, dst.) sangat penting.  
  **Catatan:** *Lag target* `Turbidity` **tidak** digunakan demi anti-leakage pada skenario one-shot inference (nilai target historis pada data uji tidak tersedia saat prediksi).
- **Fitur Temporal**: Dapat mengekstrak sinyal waktu (hour-of-day, day-of-week, month/season) dari kolom `Timestamp` untuk menangkap pola musiman/siklus. (Belum diaktifkan di notebook ini — bisa ditambahkan sebagai opsi lanjutan.)
- **Fitur Interaksi**: Opsional, misal kombinasi `Average Water Speed × Salinity` atau `Temperature × DO`, jika secara domain dianggap relevan.

## Regularisasi
- Model linear: gunakan **L1/L2**; Deep Learning: gunakan **Dropout**.
- Pada **LightGBM**, regularisasi dan kontrol kompleksitas diterapkan via:
  - `lambda_l2={lgb_params.get('lambda_l2', None)}`, 
  - `num_leaves={lgb_params.get('num_leaves', None)}`, 
  - `min_data_in_leaf={lgb_params.get('min_data_in_leaf', None)}`, 
  - `feature_fraction={lgb_params.get('feature_fraction', None)}`, 
  - `bagging_fraction={lgb_params.get('bagging_fraction', None)}`.

## Mengapa MSE “keras” terhadap outlier?
- MSE mengkuadratkan galat, sehingga **kesalahan besar (outlier)** mendapat penalti jauh lebih besar.  
  Pengendalian dilakukan lewat: **capping/log-transform** (bila relevan), desain **imputasi median berbasis masa lalu** (meredam lonjakan sesaat), dan validasi time-series yang ketat (gap ≥ jendela terpanjang).
"""
notes_path = OUT_DIR / "modeling_notes.md"
with open(notes_path, "w", encoding="utf-8") as f:
    f.write(notes_md)

# --- 12.3 Update metadata agar mereferensikan dokumen ini ---
try:
    # Baca metadata yang sudah dibuat di Blok 11, lalu perbarui
    meta_path = OUT_DIR / "metadata.json"
    if meta_path.exists():
        with open(meta_path, "r", encoding="utf-8") as f:
            meta_loaded = json.load(f)
    else:
        meta_loaded = {}
    meta_loaded.setdefault("docs", {})
    meta_loaded["docs"].update({
        "mse_controls_csv": str(controls_path.name),
        "modeling_notes_md": str(notes_path.name),
    })
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta_loaded, f, indent=2, ensure_ascii=False)
except Exception as e:
    print("⚠️ Gagal mengupdate metadata dengan dokumen:", e)

print("📄 Dokumen tersimpan:", controls_path.name, "dan", notes_path.name)
display_cols = list(controls_df.columns)
print(controls_df[display_cols])


📄 Dokumen tersimpan: mse_controls.csv dan modeling_notes.md
                        Area  \
0         Penanganan Outlier   
1  Penanganan Missing Values   
2  Normalisasi/Standardisasi   

                                 Teknik Pengendalian  \
0       Hapus, Transformasi Logaritma, atau Capping.   
1  Imputasi (dengan rata-rata/median), Imputasi D...   
2      Min-Max Scaling atau Z-score Standardization.   

                                          Keterangan  
0  Kunci penting untuk MSE, karena outlier mengha...  
1  Data yang hilang harus ditangani agar tidak me...  
2  Memastikan semua fitur berkontribusi secara pr...  


# mmm

In [17]:
# =========================================================
# Blok 0 — Setup, Paths, Library
# =========================================================
import os, gc, math, json, warnings, io
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# LightGBM
try:
    import lightgbm as lgb
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "lightgbm"])
    import lightgbm as lgb

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

try:
    import joblib
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "joblib"])
    import joblib

# ====== Paths (Kaggle) ======
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback lokal
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
np.random.seed(SEED)

TARGET = "Turbidity"
IDCOL  = "Record number"

# Fitur top-korelasi yang dipakai
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]

# === Opsi peningkatan ===
USE_LOG_TARGET   = True     # latih di log1p(Turbidity), inverse saat evaluasi & submit
USE_TARGET_LAGS  = False    # OFF (one-shot inference); ON butuh inferensi autoregresif
GAP_MULTIPLIER   = 1        # 1 → 24h; 2 → 48h
Q_LOW, Q_HIGH    = 0.005, 0.995  # capping quantiles (train-only)


# =========================================================
# Blok 1 — Load, Parse Timestamp, Sort, Sanity Checks
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

need_train = TOP_CORR_RAW + [TARGET, "Timestamp", IDCOL]
need_test  = TOP_CORR_RAW + ["Timestamp", IDCOL]
mt = [c for c in need_train if c not in train.columns]
ms = [c for c in need_test  if c not in test.columns]
if mt: print("⚠️ Missing train cols:", mt)
if ms: print("⚠️ Missing test cols :", ms)


# =========================================================
# Blok 2b — Outlier Capping (train-only bounds)
# =========================================================
def compute_caps(ref_df, cols, q_low, q_high):
    bounds = {}
    for c in cols:
        lo, hi = ref_df[c].quantile([q_low, q_high])
        if not np.isfinite(lo): lo = ref_df[c].min()
        if not np.isfinite(hi): hi = ref_df[c].max()
        bounds[c] = (lo, hi)
    return bounds

def apply_caps(df, bounds):
    out = df.copy()
    for c, (lo, hi) in bounds.items():
        if c in out.columns:
            out[c] = out[c].clip(lo, hi)
    return out

# Hitung batas dari TRAIN, terapkan ke TRAIN & TEST
caps = compute_caps(train, TOP_CORR_RAW, Q_LOW, Q_HIGH)
train = apply_caps(train, caps)
test  = apply_caps(test, caps)


# =========================================================
# Blok 3 — Flag Missingness (FULL RAW, sebelum imputasi)
# =========================================================
train["_seg"] = "train"
test["_seg"]  = "test"
full_raw = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

for c in TOP_CORR_RAW:
    full_raw[c+"_was_na"] = full_raw[c].isna().astype(int)

train_flags = full_raw.loc[full_raw["_seg"]=="train", [c+"_was_na" for c in TOP_CORR_RAW]].reset_index(drop=True)
test_flags  = full_raw.loc[full_raw["_seg"]=="test",  [c+"_was_na" for c in TOP_CORR_RAW]].reset_index(drop=True)



# =========================================================
# Blok 4 — Imputasi TRAIN (TRAIN-only)
# =========================================================
train["pH"] = train["pH"].clip(0, 14)
train_medians = {c: train[c].median(skipna=True) for c in TOP_CORR_RAW}

def impute_train_timeaware(df, cols, roll_win, ffill_limit, med_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        past_med = s.shift(1).rolling(window=roll_win, min_periods=1).median()  # masa lalu
        s = s.where(~s.isna(), past_med)
        s = s.ffill(limit=ffill_limit)  # ≤ 6 jam
        s = s.fillna(med_map.get(c, np.nanmedian(out[c].values)))  # fallback train-only
        out[c] = s
    return out

train_imp = impute_train_timeaware(
    df=train, cols=TOP_CORR_RAW, roll_win=steps_6h, ffill_limit=steps_6h, med_map=train_medians
)

last_known_from_train = {}
for c in TOP_CORR_RAW:
    last_val = train_imp[c].iloc[-1]
    if pd.isna(last_val):
        last_val = train_medians[c]
    last_known_from_train[c] = last_val


# =========================================================
# Blok 5 — Imputasi TEST (seed dari TRAIN, tanpa statistik test)
# =========================================================
test["pH"] = test["pH"].clip(0, 14)

def impute_test_seeded(df, cols, ffill_limit, med_map, seed_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        if len(s)>0 and pd.isna(s.iloc[0]) and pd.notna(seed_map.get(c, np.nan)):
            s.iloc[0] = seed_map[c]
        s = s.ffill(limit=ffill_limit)
        s = s.fillna(med_map.get(c, np.nanmedian(out[c].values)))
        out[c] = s
    return out

test_imp = impute_test_seeded(
    df=test, cols=TOP_CORR_RAW, ffill_limit=steps_6h, med_map=train_medians, seed_map=last_known_from_train
)




# =========================================================
# Blok 6 — Gabungkan, Tambahkan Flags, Fitur Temporal & Interaksi
# =========================================================
train_imp["_seg"] = "train"
test_imp["_seg"]  = "test"
full_imp = pd.concat([train_imp, test_imp], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

# flags
flag_cols = [c+"_was_na" for c in TOP_CORR_RAW]
full_flags = pd.concat([train_flags, test_flags], axis=0, ignore_index=True)
for col in flag_cols:
    full_imp[col] = full_flags[col].values

# Fitur temporal
def add_time_features(df):
    out = df.copy()
    out["hour"]  = out["Timestamp"].dt.hour
    out["dow"]   = out["Timestamp"].dt.dayofweek
    out["month"] = out["Timestamp"].dt.month
    # sin/cos
    out["hour_sin"] = np.sin(2*np.pi*out["hour"]/24.0)
    out["hour_cos"] = np.cos(2*np.pi*out["hour"]/24.0)
    out["dow_sin"]  = np.sin(2*np.pi*out["dow"]/7.0)
    out["dow_cos"]  = np.cos(2*np.pi*out["dow"]/7.0)
    return out

full_imp = add_time_features(full_imp)
TIME_FEATS = ["hour","dow","month","hour_sin","hour_cos","dow_sin","dow_cos"]

# Fitur interaksi (domain)
def add_interactions(df):
    out = df.copy()
    out["speed_x_sal"] = out["Average Water Speed"] * out["Salinity"]
    out["temp_x_do"]   = out["Temperature"] * out["Dissolved Oxygen"]
    out["ph_x_cond"]   = out["pH"] * out["Specific Conductance"]
    out["cond_x_sal"]  = out["Specific Conductance"] * out["Salinity"]
    return out

full_imp = add_interactions(full_imp)
INTER_FEATS = ["speed_x_sal","temp_x_do","ph_x_cond","cond_x_sal"]



# =========================================================
# Blok 6b — Lag & Rolling (sensor), Z-Score features
# =========================================================
def add_lag_features(df, cols, lags):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
    return out

def add_roll_features(df, cols, windows):
    out = df.copy()
    for c in cols:
        for w in windows:
            past = out[c].shift(1).rolling(window=w, min_periods=1)  # masa lalu
            out[f"{c}_roll{w}_mean"] = past.mean()
            out[f"{c}_roll{w}_min"]  = past.min()
            out[f"{c}_roll{w}_max"]  = past.max()
    return out

LAGS  = [1, steps_1h, steps_6h]
ROLLS = [steps_1h, steps_6h, steps_12h, steps_24h, steps_48h]

full_imp = add_lag_features(full_imp, TOP_CORR_RAW, LAGS)
full_imp = add_roll_features(full_imp, TOP_CORR_RAW, ROLLS)

lag_feats   = [c for c in full_imp.columns if "_lag" in c]
roll_feats  = [c for c in full_imp.columns if "_roll" in c and c.endswith(("mean","min","max"))]
miss_flags  = [c+"_was_na" for c in TOP_CORR_RAW]

# Z-score untuk 8 sensor (train-only mean/std)
train_mask = full_imp["_seg"]=="train"
z_means = {c: full_imp.loc[train_mask, c].mean() for c in TOP_CORR_RAW}
z_stds  = {c: full_imp.loc[train_mask, c].std(ddof=0) for c in TOP_CORR_RAW}

for c in TOP_CORR_RAW:
    m, s = z_means[c], z_stds[c]
    full_imp[c+"_z"] = (full_imp[c]-m) / (s if (s and s!=0) else 1.0)

Z_FEATS = [c+"_z" for c in TOP_CORR_RAW]


# =========================================================
# Blok 6c — (Opsional) Lag Target (OFF by default, AR inference needed)
# =========================================================
TARGET_LAGS = []
if USE_TARGET_LAGS:
    # Buat lag target untuk TRAIN saja (nanti inference test perlu AR loop)
    for L in [1, steps_1h, steps_6h]:
        full_imp[f"{TARGET}_lag{L}"] = np.nan
    # Isi untuk segmen train dengan Turbidity historis
    train_idx = full_imp["_seg"]=="train"
    for L in [1, steps_1h, steps_6h]:
        full_imp.loc[train_idx, f"{TARGET}_lag{L}"] = train[TARGET].shift(L).values
    TARGET_LAGS = [f"{TARGET}_lag{L}" for L in [1, steps_1h, steps_6h]]




# =========================================================
# Blok 7 — Split Train/Test Matrices (drop NaN awal lag/roll)
# =========================================================
train_fe = full_imp[full_imp["_seg"]=="train"].copy()
test_fe  = full_imp[full_imp["_seg"]=="test"].copy()

# Align target
train_fe[TARGET] = train[TARGET].values

# Buang baris awal train yang masih NaN (efek lag/rolling/target_lag)
feature_blocks = TOP_CORR_RAW + miss_flags + TIME_FEATS + INTER_FEATS + Z_FEATS + lag_feats + roll_feats + TARGET_LAGS
FEATURES = feature_blocks.copy()

train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

# Log target (opsional)
y_raw = train_fe[TARGET].values
y = np.log1p(y_raw) if USE_LOG_TARGET else y_raw

train_X = train_fe[FEATURES].copy()
test_X  = test_fe[FEATURES].copy()

# Isi NaN sisa test dengan median TRAIN (aman)
test_X = test_X.fillna(train_X.median())

print("n_features:", len(FEATURES))
print("train_X:", train_X.shape, "test_X:", test_X.shape, "| USE_LOG_TARGET:", USE_LOG_TARGET, "| USE_TARGET_LAGS:", USE_TARGET_LAGS)



# =========================================================
# Blok 8 — TimeSeriesSplit CV (gap = 24h) & LGBM Training (regularized)
# =========================================================
gap = steps_24h * GAP_MULTIPLIER
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

lgb_params = dict(
    objective="l2",            # MSE (di ruang log jika USE_LOG_TARGET=True)
    learning_rate=0.045,
    num_leaves=96,
    max_depth=-1,
    min_data_in_leaf=80,
    feature_fraction=0.9,
    bagging_fraction=0.8,
    bagging_freq=1,
    reg_alpha=0.2,
    reg_lambda=4.0,
    n_estimators=8000,         # besar; early_stopping memangkas
    random_state=SEED,
    verbose=-1
)

oof_pred_log = np.zeros(len(train_fe))
test_fold_preds_log = []
best_iters = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=600, verbose=False)]
    )

    best_it = model.best_iteration_
    best_iters.append(int(best_it) if best_it is not None else int(lgb_params["n_estimators"]))

    pred_va_log = model.predict(X_va, num_iteration=best_it)
    oof_pred_log[va_idx] = pred_va_log

    pred_te_log = model.predict(test_X, num_iteration=best_it)
    test_fold_preds_log.append(pred_te_log)

    # MSE OOF ruang asli
    y_va_raw = np.expm1(y_va) if USE_LOG_TARGET else y_va
    pred_va_raw = np.expm1(pred_va_log) if USE_LOG_TARGET else pred_va_log
    pred_va_raw = np.clip(pred_va_raw, 0, None)
    mse_fold = mean_squared_error(y_va_raw, pred_va_raw)
    print(f"Fold {fold} — best_iter={best_it}, MSE(raw)={mse_fold:.6f}")

best_iter_final = int(np.median(best_iters)) if len(best_iters)>0 else int(lgb_params["n_estimators"])

# CV MSE (OOF) ruang asli
oof_pred_raw = np.expm1(oof_pred_log) if USE_LOG_TARGET else oof_pred_log
oof_pred_raw = np.clip(oof_pred_raw, 0, None)
cv_mse = mean_squared_error(train_fe[TARGET].values, oof_pred_raw)
print(f"\nCV MSE (OOF, raw) = {cv_mse:.6f} | median best_iter = {best_iter_final}")

# =========================================================
# Blok 9 — Train Full Model & Prediksi Test (Full + CV-Ensemble)
# =========================================================
lgb_full = lgb.LGBMRegressor(**{**lgb_params, "n_estimators": best_iter_final})
lgb_full.fit(train_X, y)

pred_test_full_log  = lgb_full.predict(test_X)
pred_test_cvens_log = np.mean(np.vstack(test_fold_preds_log), axis=0) if len(test_fold_preds_log)>0 else pred_test_full_log

# Inverse log & clip
pred_test_full  = np.expm1(pred_test_full_log)  if USE_LOG_TARGET else pred_test_full_log
pred_test_cvens = np.expm1(pred_test_cvens_log) if USE_LOG_TARGET else pred_test_cvens_log
pred_test_full  = np.clip(pred_test_full,  0, None)
pred_test_cvens = np.clip(pred_test_cvens, 0, None)

print("Pred test (full)  mean/std:", float(pred_test_full.mean()),  float(pred_test_full.std()))
print("Pred test (cvens) mean/std:", float(pred_test_cvens.mean()), float(pred_test_cvens.std()))



# =========================================================
# Blok 10 — Submission (default: CV-ensemble)
# =========================================================
FINAL_PRED = pred_test_cvens  # ganti ke pred_test_full bila ingin full-model

submission = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": FINAL_PRED
})

if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))




# =========================================================
# Blok 11 — Artefak (model, fitur, OOF, FI, metadata)
# =========================================================
OUT_DIR = Path("./outputs_lgbm_maximized")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Submission alternatif
submission.to_csv(OUT_DIR / "submission.csv", index=False)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": pred_test_full}).to_csv(
    OUT_DIR / "submission_full_model.csv", index=False
)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": FINAL_PRED}).to_csv(
    OUT_DIR / "submission_cv_ensemble.csv", index=False
)

# Simpan model
lgb_full.booster_.save_model(str(OUT_DIR / "lgbm_full.txt"))
joblib.dump(lgb_full, OUT_DIR / "lgbm_full.pkl")

# Fitur
pd.Series(train_X.columns).to_csv(OUT_DIR / "features.txt", index=False, header=False)

# OOF
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_pred_raw[:len(oof_df)]
oof_df.to_csv(OUT_DIR / "oof_predictions.csv", index=False)

# Feature importance
fi = pd.DataFrame({
    "feature": train_X.columns,
    "gain":   lgb_full.booster_.feature_importance(importance_type="gain"),
    "split":  lgb_full.booster_.feature_importance(importance_type="split"),
}).sort_values("gain", ascending=False)
fi.to_csv(OUT_DIR / "feature_importance.csv", index=False)
print("Top 20 FI:")
print(fi.head(20))

# Metadata + caps bounds
meta = {
    "cv_mse_raw": float(cv_mse),
    "n_splits": int(n_splits),
    "gap_steps": int(gap),
    "gap_hours": float(gap / steps_per_hour),
    "lgb_params": {**lgb_params, "n_estimators": int(best_iter_final)},
    "steps_per_hour": int(steps_per_hour),
    "n_features": int(len(train_X.columns)),
    "use_log_target": bool(USE_LOG_TARGET),
    "use_target_lags": bool(USE_TARGET_LAGS),
    "caps_quantiles": [Q_LOW, Q_HIGH],
    "caps_bounds": {k: [float(v[0]), float(v[1])] for k, v in caps.items()},
    "impute_policy": {
        "train": "past_rolling_median -> ffill<=6h -> fallback median TRAIN-only",
        "test":  "seed last-known(train) -> ffill<=6h -> fallback median TRAIN-only",
    }
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)


# =========================================================
# Blok 12 — Dokumentasi (Kontrol MSE & Catatan Modeling)
# =========================================================
controls_csv = """Area,Teknik Pengendalian,Keterangan
Penanganan Outlier,"Hapus, Transformasi Logaritma, atau Capping.","Kunci penting untuk MSE, karena outlier menghasilkan kesalahan besar yang sangat memengaruhi nilai MSE."
Penanganan Missing Values,"Imputasi (dengan rata-rata/median), Imputasi Deret Waktu (Interpolasi).","Data yang hilang harus ditangani agar tidak mengganggu pola dalam deret waktu."
Normalisasi/Standardisasi,"Min-Max Scaling atau Z-score Standardization.","Memastikan semua fitur berkontribusi secara proporsional, mencegah fitur dengan skala besar mendominasi proses pelatihan."
"""
controls_df = pd.read_csv(io.StringIO(controls_csv))
controls_path = OUT_DIR / "mse_controls.csv"
controls_df.to_csv(controls_path, index=False)

notes_md = f"""# Catatan Modeling — LGBM Turbidity (versi maksimal)

## Rekap Penerapan
- **Outlier capping (train-only)**: quantile [{Q_LOW:.3f}, {Q_HIGH:.3f}] → diterapkan ke train & test.
- **Imputasi hati-hati**: rolling-median masa lalu → ffill≤6h → fallback median train-only.
- **Normalisasi/Standardisasi**: fitur **z-score** untuk 8 sensor (train mean/std).
- **Fitur Temporal**: hour/dow/month + sin/cos.
- **Interaksi**: speed×salinity, temp×DO, pH×conductance, conductance×salinity.
- **Lag/Roll (sensor)**: 1-step, 1h, 6h, & rolling 1h/6h/12h/24h/48h (mean/min/max).
- **Log-target**: {USE_LOG_TARGET}.
- **Target lags** (opsional): {USE_TARGET_LAGS} (aktif → butuh AR inference).

## Kenapa MSE keras pada outlier?
MSE mengkuadratkan galat → kesalahan besar dihukum jauh lebih berat.
Kita redam dengan: capping, imputasi median masa lalu, & transformasi log-target (opsional).

## Regularisasi di LGBM
- `num_leaves={lgb_params['num_leaves']}`, `min_data_in_leaf={lgb_params['min_data_in_leaf']}`,
  `reg_alpha={lgb_params['reg_alpha']}`, `reg_lambda={lgb_params['reg_lambda']}`,
  `feature_fraction={lgb_params['feature_fraction']}`, `bagging_fraction={lgb_params['bagging_fraction']}`.
"""
notes_path = OUT_DIR / "modeling_notes.md"
with open(notes_path, "w", encoding="utf-8") as f:
    f.write(notes_md)

# Update metadata
meta_path = OUT_DIR / "metadata.json"
try:
    with open(meta_path, "r", encoding="utf-8") as f:
        meta_loaded = json.load(f)
except:
    meta_loaded = {}
meta_loaded.setdefault("docs", {})
meta_loaded["docs"].update({
    "mse_controls_csv": str(controls_path.name),
    "modeling_notes_md": str(notes_path.name),
})
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(meta_loaded, f, indent=2, ensure_ascii=False)

print("📄 Dokumen tersimpan:", controls_path.name, "dan", notes_path.name)
print(controls_df)


n_features: 179
train_X: (47346, 179) test_X: (14610, 179) | USE_LOG_TARGET: True | USE_TARGET_LAGS: False
Fold 1 — best_iter=546, MSE(raw)=2.524500
Fold 2 — best_iter=4545, MSE(raw)=38.570857
Fold 3 — best_iter=7, MSE(raw)=12.862864
Fold 4 — best_iter=7, MSE(raw)=9.556301
Fold 5 — best_iter=267, MSE(raw)=43.463531

CV MSE (OOF, raw) = 18.690772 | median best_iter = 267
Pred test (full)  mean/std: 6.378930636974528 3.163879498663481
Pred test (cvens) mean/std: 4.669245304047889 0.6604283203701171
✅ submission.csv dibuat!
   Record number  Turbidity
0          54916   3.572625
1          54917   3.592873
2          54918   3.655010
3          54919   3.598167
4          54920   3.688808
5          54921   3.619780
6          54922   3.642764
7          54923   3.719023
8          54924   3.682903
9          54925   4.245930
Top 20 FI:
                                        feature          gain  split
73             Specific Conductance_roll288_max  19207.092807    179
83             A

In [19]:

# =========================================================
# Blok 0 — Setup, Paths, Library
# =========================================================
import os, gc, math, json, warnings, io
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# LightGBM
try:
    import lightgbm as lgb
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "lightgbm"])
    import lightgbm as lgb

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

try:
    import joblib
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "joblib"])
    import joblib

# ====== Paths (Kaggle) ======
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback lokal
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
np.random.seed(SEED)

TARGET = "Turbidity"
IDCOL  = "Record number"

# Fitur sensor top-korelasi (raw)
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]

# === Opsi peningkatan ===
USE_LOG_TARGET   = True      # latih di log1p(Turbidity), inverse saat evaluasi & submit
USE_TARGET_LAGS  = False     # OFF (one-shot inference). ON => butuh AR inference
USE_DIR_FEATURES = True      # pakai arah arus → sin/cos + u/v
ADD_DELTAS       = True      # tambah fitur perubahan (d1, d6h)
ADD_EWM          = True      # tambah EWMA (6h, 24h) anti-leakage
CV_BLEND         = "median"  # 'mean' atau 'median' untuk blending prediksi CV
BOOSTING_TYPE    = "gbdt"    # 'gbdt' atau 'dart'
GAP_MULTIPLIER   = 1         # 1→24h, 2→48h gap CV
Q_LOW, Q_HIGH    = 0.005, 0.995  # capping quantiles (train-only)

DIR_COL = "Average Water Direction"  # arah arus (derajat)


# =========================================================
# Blok 1 — Load, Parse Timestamp, Sort, Sanity Checks
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

need_train = TOP_CORR_RAW + ([DIR_COL] if DIR_COL in train.columns else []) + [TARGET, "Timestamp", IDCOL]
need_test  = TOP_CORR_RAW + ([DIR_COL] if DIR_COL in test.columns  else []) + ["Timestamp", IDCOL]
mt = [c for c in need_train if c not in train.columns]
ms = [c for c in need_test  if c not in test.columns]
if mt: print("⚠️ Missing train cols:", mt)
if ms: print("⚠️ Missing test cols :", ms)


# =========================================================
# Blok 1a — Drop baris dengan TARGET NaN (train-only)
# =========================================================
# Pastikan target numerik; string seperti "NaN" akan jadi NaN
train[TARGET] = pd.to_numeric(train[TARGET], errors="coerce")

n_before = len(train)
train = train[train[TARGET].notna()].reset_index(drop=True)
n_dropped = n_before - len(train)
print(f"🧹 Dropped {n_dropped} rows with NaN in '{TARGET}' from train.")




# =========================================================
# Blok 2 — Estimasi Resolusi Waktu (steps/hour)
# =========================================================
def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt)==0: return 1
    med = np.median(dt)
    if not np.isfinite(med) or med<=0: return 1
    return int(max(1, round(3600.0/med)))

steps_per_hour = estimate_steps_per_hour(train)
steps_1h  = max(1, steps_per_hour)
steps_6h  = max(1, 6*steps_per_hour)
steps_12h = max(1, 12*steps_per_hour)
steps_24h = max(1, 24*steps_per_hour)
steps_48h = max(1, 48*steps_per_hour)

print(f"steps/hour={steps_per_hour} | 1h={steps_1h} 6h={steps_6h} 12h={steps_12h} 24h={steps_24h} 48h={steps_48h}")


# =========================================================
# Blok 2b — Outlier Capping (train-only bounds untuk 8 sensor)
# =========================================================
def compute_caps(ref_df, cols, q_low, q_high):
    bounds = {}
    for c in cols:
        lo, hi = ref_df[c].quantile([q_low, q_high])
        if not np.isfinite(lo): lo = ref_df[c].min()
        if not np.isfinite(hi): hi = ref_df[c].max()
        bounds[c] = (lo, hi)
    return bounds

def apply_caps(df, bounds):
    out = df.copy()
    for c, (lo, hi) in bounds.items():
        if c in out.columns:
            out[c] = out[c].clip(lo, hi)
    return out

caps = compute_caps(train, TOP_CORR_RAW, Q_LOW, Q_HIGH)
train = apply_caps(train, caps)
test  = apply_caps(test,  caps)



# =========================================================
# Blok 3 — Flag Missingness (FULL RAW, sebelum imputasi)
# =========================================================
# Daftar kolom yang diimputasi (sensor + arah jika ada)
IMP_COLS = TOP_CORR_RAW + ([DIR_COL] if (USE_DIR_FEATURES and DIR_COL in train.columns) else [])

train["_seg"] = "train"
test["_seg"]  = "test"
full_raw = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

for c in IMP_COLS:
    full_raw[c+"_was_na"] = full_raw[c].isna().astype(int)

train_flags = full_raw.loc[full_raw["_seg"]=="train", [c+"_was_na" for c in IMP_COLS]].reset_index(drop=True)
test_flags  = full_raw.loc[full_raw["_seg"]=="test",  [c+"_was_na" for c in IMP_COLS]].reset_index(drop=True)


# =========================================================
# Blok 4 — Imputasi TRAIN (TRAIN-only)
# =========================================================
train["pH"] = train["pH"].clip(0, 14)
if USE_DIR_FEATURES and DIR_COL in train.columns:
    train[DIR_COL] = (train[DIR_COL] % 360).clip(0, 360)

train_medians = {c: train[c].median(skipna=True) for c in IMP_COLS}

def impute_train_timeaware(df, cols, roll_win, ffill_limit, med_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        past_med = s.shift(1).rolling(window=roll_win, min_periods=1).median()  # masa lalu saja
        s = s.where(~s.isna(), past_med)
        s = s.ffill(limit=ffill_limit)  # ≤ 6 jam
        s = s.fillna(med_map.get(c, np.nanmedian(out[c].values)))  # fallback train-only
        out[c] = s
    return out

train_imp = impute_train_timeaware(
    df=train, cols=IMP_COLS, roll_win=steps_6h, ffill_limit=steps_6h, med_map=train_medians
)

last_known_from_train = {}
for c in IMP_COLS:
    last_val = train_imp[c].iloc[-1]
    if pd.isna(last_val):
        last_val = train_medians[c]
    last_known_from_train[c] = last_val



# =========================================================
# Blok 5 — Imputasi TEST (seed dari TRAIN, tanpa statistik test)
# =========================================================
test["pH"] = test["pH"].clip(0, 14)
if USE_DIR_FEATURES and DIR_COL in test.columns:
    test[DIR_COL] = (test[DIR_COL] % 360).clip(0, 360)

def impute_test_seeded(df, cols, ffill_limit, med_map, seed_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        if len(s)>0 and pd.isna(s.iloc[0]) and pd.notna(seed_map.get(c, np.nan)):
            s.iloc[0] = seed_map[c]
        s = s.ffill(limit=ffill_limit)  # ≤ 6 jam
        s = s.fillna(med_map.get(c, np.nanmedian(out[c].values)))  # fallback median TRAIN-only
        out[c] = s
    return out

test_imp = impute_test_seeded(
    df=test, cols=IMP_COLS, ffill_limit=steps_6h, med_map=train_medians, seed_map=last_known_from_train
)



# =========================================================
# Blok 6 — Gabungkan, Tambahkan Flags, Fitur Temporal & Interaksi
# =========================================================
train_imp["_seg"] = "train"
test_imp["_seg"]  = "test"
full_imp = pd.concat([train_imp, test_imp], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

# flags
flag_cols = [c+"_was_na" for c in IMP_COLS]
full_flags = pd.concat([train_flags, test_flags], axis=0, ignore_index=True)
for col in flag_cols:
    full_imp[col] = full_flags[col].values

# Direction → sin/cos + u/v
DIR_FEATS = []
if USE_DIR_FEATURES and DIR_COL in full_imp.columns:
    full_imp["dir_rad"] = np.deg2rad((full_imp[DIR_COL] % 360).fillna(0.0))
    full_imp["dir_sin"] = np.sin(full_imp["dir_rad"])
    full_imp["dir_cos"] = np.cos(full_imp["dir_rad"])
    full_imp["u"] = full_imp["Average Water Speed"] * full_imp["dir_cos"]
    full_imp["v"] = full_imp["Average Water Speed"] * full_imp["dir_sin"]
    DIR_FEATS = ["dir_sin","dir_cos","u","v"]

# Fitur temporal
def add_time_features(df):
    out = df.copy()
    out["hour"]  = out["Timestamp"].dt.hour
    out["dow"]   = out["Timestamp"].dt.dayofweek
    out["month"] = out["Timestamp"].dt.month
    # sin/cos siklik
    out["hour_sin"] = np.sin(2*np.pi*out["hour"]/24.0)
    out["hour_cos"] = np.cos(2*np.pi*out["hour"]/24.0)
    out["dow_sin"]  = np.sin(2*np.pi*out["dow"]/7.0)
    out["dow_cos"]  = np.cos(2*np.pi*out["dow"]/7.0)
    return out

full_imp = add_time_features(full_imp)
TIME_FEATS = ["hour","dow","month","hour_sin","hour_cos","dow_sin","dow_cos"]

# Fitur interaksi (domain)
def add_interactions(df):
    out = df.copy()
    out["speed_x_sal"] = out["Average Water Speed"] * out["Salinity"]
    out["temp_x_do"]   = out["Temperature"] * out["Dissolved Oxygen"]
    out["ph_x_cond"]   = out["pH"] * out["Specific Conductance"]
    out["cond_x_sal"]  = out["Specific Conductance"] * out["Salinity"]
    return out

full_imp = add_interactions(full_imp)
INTER_FEATS = ["speed_x_sal","temp_x_do","ph_x_cond","cond_x_sal"]



# =========================================================
# Blok 6b — Lag & Rolling (sensor + u/v), Z-Score, Delta & EWMA
# =========================================================
def add_lag_features(df, cols, lags):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
    return out

def add_roll_features(df, cols, windows):
    out = df.copy()
    for c in cols:
        for w in windows:
            past = out[c].shift(1).rolling(window=w, min_periods=1)  # masa lalu
            out[f"{c}_roll{w}_mean"] = past.mean()
            out[f"{c}_roll{w}_min"]  = past.min()
            out[f"{c}_roll{w}_max"]  = past.max()
    return out

# Sensor utama untuk lag/roll/z
SENSOR_COLS = TOP_CORR_RAW + (["u","v"] if ("u" in full_imp.columns and "v" in full_imp.columns) else [])

# Lag & rolling
full_imp = add_lag_features(full_imp, SENSOR_COLS, [1, steps_1h, steps_6h])
full_imp = add_roll_features(full_imp, SENSOR_COLS, [steps_1h, steps_6h, steps_12h, steps_24h, steps_48h])

lag_feats   = [c for c in full_imp.columns if "_lag"  in c]
roll_feats  = [c for c in full_imp.columns if "_roll" in c and c.endswith(("mean","min","max"))]
miss_flags  = [c+"_was_na" for c in IMP_COLS]

# Z-score (train-only mean/std)
train_mask = full_imp["_seg"]=="train"
Z_FEATS = []
for c in SENSOR_COLS:
    m = full_imp.loc[train_mask, c].mean()
    s = full_imp.loc[train_mask, c].std(ddof=0)
    full_imp[c+"_z"] = (full_imp[c]-m) / (s if (s and s!=0) else 1.0)
    Z_FEATS.append(c+"_z")

# Delta features
DELTA_FEATS = []
if ADD_DELTAS:
    for c in SENSOR_COLS:
        full_imp[f"{c}_d1"]   = full_imp[c] - full_imp[c].shift(1)
        full_imp[f"{c}_d6h"]  = full_imp[c] - full_imp[c].shift(steps_6h)
        DELTA_FEATS += [f"{c}_d1", f"{c}_d6h"]

# EWMA (halflife ~ window; anti-leakage via shift(1))
EWM_FEATS = []
if ADD_EWM:
    for c in SENSOR_COLS:
        s = full_imp[c].shift(1)
        full_imp[f"{c}_ewm6_mean"]  = s.ewm(halflife=steps_6h,  min_periods=1).mean()
        full_imp[f"{c}_ewm24_mean"] = s.ewm(halflife=steps_24h, min_periods=1).mean()
        EWM_FEATS += [f"{c}_ewm6_mean", f"{c}_ewm24_mean"]




# =========================================================
# Blok 6c — (Opsional) Lag Target (OFF by default, AR inference needed)
# =========================================================
TARGET_LAGS = []
if USE_TARGET_LAGS:
    for L in [1, steps_1h, steps_6h]:
        full_imp[f"{TARGET}_lag{L}"] = np.nan
    train_idx = full_imp["_seg"]=="train"
    for L in [1, steps_1h, steps_6h]:
        full_imp.loc[train_idx, f"{TARGET}_lag{L}"] = train[TARGET].shift(L).values
    TARGET_LAGS = [f"{TARGET}_lag{L}" for L in [1, steps_1h, steps_6h]]



# =========================================================
# Blok 7 — Split Train/Test Matrices (drop NaN awal lag/roll)
# =========================================================
train_fe = full_imp[full_imp["_seg"]=="train"].copy()
test_fe  = full_imp[full_imp["_seg"]=="test"].copy()

# Align target
train_fe[TARGET] = train[TARGET].values

# Kumpulkan fitur
TIME_FEATS = ["hour","dow","month","hour_sin","hour_cos","dow_sin","dow_cos"]
FEATURES = (
    TOP_CORR_RAW +              # sensor mentah
    DIR_FEATS +                 # sin/cos, u/v (jika ada)
    miss_flags +                # flags NA
    TIME_FEATS +                # temporal
    INTER_FEATS +              # interaksi
    Z_FEATS +                   # z-score
    lag_feats + roll_feats +    # lag & rolling
    DELTA_FEATS + EWM_FEATS +   # delta & ewma
    TARGET_LAGS                 # target lag (opsional)
)

# Drop baris awal train yang masih NaN (efek lag/roll/ewm)
train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

# Log target (opsional)
y_raw = train_fe[TARGET].values
y = np.log1p(y_raw) if USE_LOG_TARGET else y_raw

train_X = train_fe[FEATURES].copy()
test_X  = test_fe[FEATURES].copy()

# Isi NaN sisa test dengan median TRAIN (aman)
test_X = test_X.fillna(train_X.median())

print("n_features:", len(FEATURES))
print("train_X:", train_X.shape, "test_X:", test_X.shape, "| USE_LOG_TARGET:", USE_LOG_TARGET, "| USE_TARGET_LAGS:", USE_TARGET_LAGS)


# =========================================================
# Blok 8 — TimeSeriesSplit CV (gap = 24h) & LGBM Training (regularized + extra_trees)
# =========================================================
gap = steps_24h * GAP_MULTIPLIER
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

lgb_params = dict(
    objective="l2",
    boosting_type=BOOSTING_TYPE,   # 'gbdt' atau 'dart'
    learning_rate=0.035,
    num_leaves=72,
    max_depth=-1,
    min_data_in_leaf=120,
    feature_fraction=0.9,
    feature_fraction_bynode=0.8,
    bagging_fraction=0.75,
    bagging_freq=1,
    extra_trees=True,
    reg_alpha=0.3,
    reg_lambda=6.0,
    n_estimators=12000,
    random_state=SEED,
    verbose=-1
)

oof_pred_log = np.zeros(len(train_fe))
test_fold_preds_log = []
best_iters = []
fold = 0

for tr_idx, va_idx in tscv.split(train_X):
    fold += 1
    X_tr, y_tr = train_X.iloc[tr_idx], y[tr_idx]
    X_va, y_va = train_X.iloc[va_idx], y[va_idx]

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=800, verbose=False)]
    )

    best_it = model.best_iteration_
    best_iters.append(int(best_it) if best_it is not None else int(lgb_params["n_estimators"]))

    pred_va_log = model.predict(X_va, num_iteration=best_it)
    oof_pred_log[va_idx] = pred_va_log

    pred_te_log = model.predict(test_X, num_iteration=best_it)
    test_fold_preds_log.append(pred_te_log)

    # MSE OOF ruang asli
    y_va_raw = np.expm1(y_va) if USE_LOG_TARGET else y_va
    pred_va_raw = np.expm1(pred_va_log) if USE_LOG_TARGET else pred_va_log
    pred_va_raw = np.clip(pred_va_raw, 0, None)
    mse_fold = mean_squared_error(y_va_raw, pred_va_raw)
    print(f"Fold {fold} — best_iter={best_it}, MSE(raw)={mse_fold:.6f}")

best_iter_final = int(np.median(best_iters)) if len(best_iters)>0 else int(lgb_params["n_estimators"])

# CV MSE (OOF) ruang asli
oof_pred_raw = np.expm1(oof_pred_log) if USE_LOG_TARGET else oof_pred_log
oof_pred_raw = np.clip(oof_pred_raw, 0, None)
cv_mse = mean_squared_error(train_fe[TARGET].values, oof_pred_raw)
print(f"\nCV MSE (OOF, raw) = {cv_mse:.6f} | median best_iter = {best_iter_final}")


# =========================================================
# Blok 9 — Train Full Model & Prediksi Test (Full + CV-Blending median)
# =========================================================
lgb_full = lgb.LGBMRegressor(**{**lgb_params, "n_estimators": best_iter_final})
lgb_full.fit(train_X, y)

pred_test_full_log  = lgb_full.predict(test_X)
stack_te = np.vstack(test_fold_preds_log) if len(test_fold_preds_log)>0 else np.vstack([pred_test_full_log])
pred_test_cvens_log_mean   = stack_te.mean(axis=0)
pred_test_cvens_log_median = np.median(stack_te, axis=0)

def inv_and_clip(x):
    return np.clip(np.expm1(x) if USE_LOG_TARGET else x, 0, None)

# Blend pilihan
if CV_BLEND == "mean":
    pred_test_cvens = inv_and_clip(pred_test_cvens_log_mean)
else:
    pred_test_cvens = inv_and_clip(pred_test_cvens_log_median)

pred_test_full = inv_and_clip(pred_test_full_log)

print("Blend:", CV_BLEND)
print("Pred test (full)  mean/std:", float(pred_test_full.mean()),  float(pred_test_full.std()))
print("Pred test (cvens) mean/std:", float(pred_test_cvens.mean()), float(pred_test_cvens.std()))



# =========================================================
# Blok 10 — Submission (default: CV-blended)
# =========================================================
FINAL_PRED = pred_test_cvens  # ganti ke pred_test_full bila ingin full-model

submission = pd.DataFrame({
    "Record number": test_fe[IDCOL].values,
    "Turbidity": FINAL_PRED
})

# Jaga urutan sesuai sample
if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))



# =========================================================
# Blok 11 — Artefak (model, fitur, OOF, FI, metadata)
# =========================================================
OUT_DIR = Path("./outputs_lgbm_maximized")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Submission alternatif
submission.to_csv(OUT_DIR / "submission.csv", index=False)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": pred_test_full}).to_csv(
    OUT_DIR / "submission_full_model.csv", index=False
)
pd.DataFrame({"Record number": test_fe[IDCOL].values, "Turbidity": FINAL_PRED}).to_csv(
    OUT_DIR / "submission_cv_ensemble.csv", index=False
)

# Simpan model
lgb_full.booster_.save_model(str(OUT_DIR / "lgbm_full.txt"))
joblib.dump(lgb_full, OUT_DIR / "lgbm_full.pkl")

# Fitur
pd.Series(train_X.columns).to_csv(OUT_DIR / "features.txt", index=False, header=False)

# OOF
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_pred_raw[:len(oof_df)]
oof_df.to_csv(OUT_DIR / "oof_predictions.csv", index=False)

# Feature importance
fi = pd.DataFrame({
    "feature": train_X.columns,
    "gain":   lgb_full.booster_.feature_importance(importance_type="gain"),
    "split":  lgb_full.booster_.feature_importance(importance_type="split"),
}).sort_values("gain", ascending=False)
fi.to_csv(OUT_DIR / "feature_importance.csv", index=False)
print("Top 20 FI:")
print(fi.head(20))

# Metadata + caps bounds
meta = {
    "cv_mse_raw": float(cv_mse),
    "n_splits": int(n_splits),
    "gap_steps": int(gap),
    "gap_hours": float(gap / steps_per_hour),
    "lgb_params": {**lgb_params, "n_estimators": int(best_iter_final)},
    "steps_per_hour": int(steps_per_hour),
    "n_features": int(len(train_X.columns)),
    "use_log_target": bool(USE_LOG_TARGET),
    "use_target_lags": bool(USE_TARGET_LAGS),
    "caps_quantiles": [Q_LOW, Q_HIGH],
    "caps_bounds": {k: [float(v[0]), float(v[1])] for k, v in caps.items()},
    "blend": CV_BLEND,
    "use_dir_features": bool(USE_DIR_FEATURES),
    "add_deltas": bool(ADD_DELTAS),
    "add_ewm": bool(ADD_EWM),
    "boosting_type": BOOSTING_TYPE,
    "impute_policy": {
        "train": "past_rolling_median -> ffill<=6h -> fallback median TRAIN-only",
        "test":  "seed last-known(train) -> ffill<=6h -> fallback median TRAIN-only",
    }
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)






🧹 Dropped 6066 rows with NaN in 'Turbidity' from train.
steps/hour=6 | 1h=6 6h=36 12h=72 24h=144 48h=288
n_features: 262
train_X: (47346, 262) test_X: (14610, 262) | USE_LOG_TARGET: True | USE_TARGET_LAGS: False
Fold 1 — best_iter=259, MSE(raw)=5.854045
Fold 2 — best_iter=108, MSE(raw)=41.684473
Fold 3 — best_iter=9, MSE(raw)=13.134942
Fold 4 — best_iter=6, MSE(raw)=9.766971
Fold 5 — best_iter=904, MSE(raw)=42.985433

CV MSE (OOF, raw) = 19.765407 | median best_iter = 108
Blend: median
Pred test (full)  mean/std: 5.908182841917406 2.2359612857142936
Pred test (cvens) mean/std: 3.4844024459642173 0.2549995364472989
✅ submission.csv dibuat!
   Record number  Turbidity
0          54916   3.109019
1          54917   3.128946
2          54918   3.128946
3          54919   3.128946
4          54920   3.128946
5          54921   3.110548
6          54922   3.128946
7          54923   3.128946
8          54924   3.128946
9          54925   3.128946
Top 20 FI:
                                  

In [20]:
# =========================================================
# Blok 0 — Setup, Paths, Library
# =========================================================
import os, gc, math, json, warnings, io
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.utils.validation import check_is_fitted
import joblib

# ====== Paths (Kaggle) ======
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback lokal
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
rng = np.random.RandomState(SEED)
np.random.seed(SEED)

TARGET = "Turbidity"
IDCOL  = "Record number"

# Sensor top-korelasi (raw)
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]

# === Opsi modeling ===
USE_LOG_TARGET   = True      # latih di log1p(Turbidity), inverse saat evaluasi & submit
USE_TARGET_LAGS  = False     # ON => akan buat fitur lag target + (opsional) AR inference
USE_DIR_FEATURES = True      # arah arus → sin/cos + u/v
ADD_DELTAS       = True      # delta d1 & d6h
ADD_EWM          = True      # EWMA 6h & 24h (anti-leakage)
GAP_MULTIPLIER   = 1         # 1→24h gap antar fold
Q_LOW, Q_HIGH    = 0.005, 0.995  # outlier capping quantiles (train-only)

# Linear model toggle
TRY_RIDGE        = True      # uji RidgeCV selain LinearRegression, pilih yang terbaik
RIDGE_ALPHAS     = np.logspace(-3, 3, 13)  # kandidat alpha Ridge

DIR_COL = "Average Water Direction"  # derajat

# =========================================================
# Blok 1 — Load, Parse Timestamp, Sort, Sanity Checks
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

# sort
train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

# pastikan kolom
need_train = TOP_CORR_RAW + ([DIR_COL] if DIR_COL in train.columns else []) + [TARGET, "Timestamp", IDCOL]
need_test  = TOP_CORR_RAW + ([DIR_COL] if DIR_COL in test.columns  else []) + ["Timestamp", IDCOL]
mt = [c for c in need_train if c not in train.columns]
ms = [c for c in need_test  if c not in test.columns]
if mt: print("⚠️ Missing train cols:", mt)
if ms: print("⚠️ Missing test cols :", ms)

# =========================================================
# Blok 1a — Drop baris dengan TARGET NaN (train-only)
# =========================================================
train[TARGET] = pd.to_numeric(train[TARGET], errors="coerce")
n_before = len(train)
train = train[train[TARGET].notna()].reset_index(drop=True)
print(f"🧹 Dropped {n_before - len(train)} rows with NaN in '{TARGET}' from train.")

# =========================================================
# Blok 2 — Estimasi Resolusi Waktu (steps/hour)
# =========================================================
def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt)==0: return 1
    med = np.median(dt)
    if not np.isfinite(med) or med<=0: return 1
    return int(max(1, round(3600.0/med)))

steps_per_hour = estimate_steps_per_hour(train)
steps_1h  = max(1, steps_per_hour)
steps_6h  = max(1, 6*steps_per_hour)
steps_12h = max(1, 12*steps_per_hour)
steps_24h = max(1, 24*steps_per_hour)
steps_48h = max(1, 48*steps_per_hour)
print(f"steps/hour={steps_per_hour} | 1h={steps_1h} 6h={steps_6h} 12h={steps_12h} 24h={steps_24h} 48h={steps_48h}")

# =========================================================
# Blok 2b — Outlier Capping (train-only bounds untuk 8 sensor)
# =========================================================
def compute_caps(ref_df, cols, q_low, q_high):
    bounds = {}
    for c in cols:
        lo, hi = ref_df[c].quantile([q_low, q_high])
        if not np.isfinite(lo): lo = ref_df[c].min()
        if not np.isfinite(hi): hi = ref_df[c].max()
        bounds[c] = (lo, hi)
    return bounds

def apply_caps(df, bounds):
    out = df.copy()
    for c, (lo, hi) in bounds.items():
        if c in out.columns:
            out[c] = out[c].clip(lo, hi)
    return out

caps = compute_caps(train, TOP_CORR_RAW, Q_LOW, Q_HIGH)
train = apply_caps(train, caps)
test  = apply_caps(test,  caps)

# =========================================================
# Blok 3 — Flag Missingness (FULL RAW, sebelum imputasi)
# =========================================================
IMP_COLS = TOP_CORR_RAW + ([DIR_COL] if (USE_DIR_FEATURES and DIR_COL in train.columns) else [])
train["_seg"] = "train"
test["_seg"]  = "test"
full_raw = pd.concat([train, test], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

for c in IMP_COLS:
    full_raw[c+"_was_na"] = full_raw[c].isna().astype(int)

train_flags = full_raw.loc[full_raw["_seg"]=="train", [c+"_was_na" for c in IMP_COLS]].reset_index(drop=True)
test_flags  = full_raw.loc[full_raw["_seg"]=="test",  [c+"_was_na" for c in IMP_COLS]].reset_index(drop=True)

# =========================================================
# Blok 4 — Imputasi TRAIN (TRAIN-only)
# =========================================================
train["pH"] = train["pH"].clip(0, 14)
if USE_DIR_FEATURES and DIR_COL in train.columns:
    train[DIR_COL] = (train[DIR_COL] % 360).clip(0, 360)

train_medians = {c: train[c].median(skipna=True) for c in IMP_COLS}

def impute_train_timeaware(df, cols, roll_win, ffill_limit, med_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        past_med = s.shift(1).rolling(window=roll_win, min_periods=1).median()
        s = s.where(~s.isna(), past_med)
        s = s.ffill(limit=ffill_limit)  # ≤ 6 jam
        s = s.fillna(med_map.get(c, np.nanmedian(out[c].values)))
        out[c] = s
    return out

train_imp = impute_train_timeaware(
    df=train, cols=IMP_COLS, roll_win=steps_6h, ffill_limit=steps_6h, med_map=train_medians
)

last_known_from_train = {}
for c in IMP_COLS:
    last_val = train_imp[c].iloc[-1]
    if pd.isna(last_val): last_val = train_medians[c]
    last_known_from_train[c] = last_val

# =========================================================
# Blok 5 — Imputasi TEST (seed dari TRAIN, tanpa statistik test)
# =========================================================
test["pH"] = test["pH"].clip(0, 14)
if USE_DIR_FEATURES and DIR_COL in test.columns:
    test[DIR_COL] = (test[DIR_COL] % 360).clip(0, 360)

def impute_test_seeded(df, cols, ffill_limit, med_map, seed_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        if len(s)>0 and pd.isna(s.iloc[0]) and pd.notna(seed_map.get(c, np.nan)):
            s.iloc[0] = seed_map[c]
        s = s.ffill(limit=ffill_limit)
        s = s.fillna(med_map.get(c, np.nanmedian(out[c].values)))
        out[c] = s
    return out

test_imp = impute_test_seeded(
    df=test, cols=IMP_COLS, ffill_limit=steps_6h, med_map=train_medians, seed_map=last_known_from_train
)

# =========================================================
# Blok 6 — Gabungkan, Flags, Fitur Temporal & Interaksi
# =========================================================
train_imp["_seg"] = "train"
test_imp["_seg"]  = "test"
full_imp = pd.concat([train_imp, test_imp], axis=0, ignore_index=True).sort_values("Timestamp").reset_index(drop=True)

# flags
flag_cols = [c+"_was_na" for c in IMP_COLS]
full_flags = pd.concat([train_flags, test_flags], axis=0, ignore_index=True)
for col in flag_cols:
    full_imp[col] = full_flags[col].values

# Direction → sin/cos + u/v
DIR_FEATS = []
if USE_DIR_FEATURES and DIR_COL in full_imp.columns:
    full_imp["dir_rad"] = np.deg2rad((full_imp[DIR_COL] % 360).fillna(0.0))
    full_imp["dir_sin"] = np.sin(full_imp["dir_rad"])
    full_imp["dir_cos"] = np.cos(full_imp["dir_rad"])
    full_imp["u"] = full_imp["Average Water Speed"] * full_imp["dir_cos"]
    full_imp["v"] = full_imp["Average Water Speed"] * full_imp["dir_sin"]
    DIR_FEATS = ["dir_sin","dir_cos","u","v"]

# Fitur temporal
def add_time_features(df):
    out = df.copy()
    out["hour"]  = out["Timestamp"].dt.hour
    out["dow"]   = out["Timestamp"].dt.dayofweek
    out["month"] = out["Timestamp"].dt.month
    out["hour_sin"] = np.sin(2*np.pi*out["hour"]/24.0)
    out["hour_cos"] = np.cos(2*np.pi*out["hour"]/24.0)
    out["dow_sin"]  = np.sin(2*np.pi*out["dow"]/7.0)
    out["dow_cos"]  = np.cos(2*np.pi*out["dow"]/7.0)
    return out

full_imp = add_time_features(full_imp)
TIME_FEATS = ["hour","dow","month","hour_sin","hour_cos","dow_sin","dow_cos"]

# Interaksi
def add_interactions(df):
    out = df.copy()
    out["speed_x_sal"] = out["Average Water Speed"] * out["Salinity"]
    out["temp_x_do"]   = out["Temperature"] * out["Dissolved Oxygen"]
    out["ph_x_cond"]   = out["pH"] * out["Specific Conductance"]
    out["cond_x_sal"]  = out["Specific Conductance"] * out["Salinity"]
    return out

full_imp = add_interactions(full_imp)
INTER_FEATS = ["speed_x_sal","temp_x_do","ph_x_cond","cond_x_sal"]

# =========================================================
# Blok 6b — Lag & Rolling (sensor + u/v), Z-Score, Delta & EWMA
# =========================================================
def add_lag_features(df, cols, lags):
    out = df.copy()
    for c in cols:
        for L in lags:
            out[f"{c}_lag{L}"] = out[c].shift(L)
    return out

def add_roll_features(df, cols, windows):
    out = df.copy()
    for c in cols:
        for w in windows:
            past = out[c].shift(1).rolling(window=w, min_periods=1)
            out[f"{c}_roll{w}_mean"] = past.mean()
            out[f"{c}_roll{w}_min"]  = past.min()
            out[f"{c}_roll{w}_max"]  = past.max()
    return out

SENSOR_COLS = TOP_CORR_RAW + (["u","v"] if ("u" in full_imp.columns and "v" in full_imp.columns) else [])
full_imp = add_lag_features(full_imp, SENSOR_COLS, [1, steps_1h, steps_6h])
full_imp = add_roll_features(full_imp, SENSOR_COLS, [steps_1h, steps_6h, steps_12h, steps_24h, steps_48h])

lag_feats   = [c for c in full_imp.columns if "_lag"  in c]
roll_feats  = [c for c in full_imp.columns if "_roll" in c and c.endswith(("mean","min","max"))]
miss_flags  = [c+"_was_na" for c in IMP_COLS]

# Z-score (train-only mean/std untuk raw sensor & u/v)
train_mask = full_imp["_seg"]=="train"
Z_FEATS = []
for c in SENSOR_COLS:
    m = full_imp.loc[train_mask, c].mean()
    s = full_imp.loc[train_mask, c].std(ddof=0)
    full_imp[c+"_z"] = (full_imp[c]-m) / (s if (s and s!=0) else 1.0)
    Z_FEATS.append(c+"_z")

# Delta
DELTA_FEATS = []
if ADD_DELTAS:
    for c in SENSOR_COLS:
        full_imp[f"{c}_d1"]   = full_imp[c] - full_imp[c].shift(1)
        full_imp[f"{c}_d6h"]  = full_imp[c] - full_imp[c].shift(steps_6h)
        DELTA_FEATS += [f"{c}_d1", f"{c}_d6h"]

# EWMA
EWM_FEATS = []
if ADD_EWM:
    for c in SENSOR_COLS:
        s = full_imp[c].shift(1)
        full_imp[f"{c}_ewm6_mean"]  = s.ewm(halflife=steps_6h,  min_periods=1).mean()
        full_imp[f"{c}_ewm24_mean"] = s.ewm(halflife=steps_24h, min_periods=1).mean()
        EWM_FEATS += [f"{c}_ewm6_mean", f"{c}_ewm24_mean"]

# =========================================================
# Blok 6c — Lag Target (opsional)
# =========================================================
TARGET_LAGS = []
if USE_TARGET_LAGS:
    for L in [1, steps_1h, steps_6h]:
        full_imp[f"{TARGET}_lag{L}"] = np.nan
    train_idx = full_imp["_seg"]=="train"
    for L in [1, steps_1h, steps_6h]:
        full_imp.loc[train_idx, f"{TARGET}_lag{L}"] = train[TARGET].shift(L).values
    TARGET_LAGS = [f"{TARGET}_lag{L}" for L in [1, steps_1h, steps_6h]]

# =========================================================
# Blok 7 — Split, Standarisasi & Matrices
# =========================================================
train_fe = full_imp[full_imp["_seg"]=="train"].copy()
test_fe  = full_imp[full_imp["_seg"]=="test"].copy()
train_fe[TARGET] = train[TARGET].values

TIME_FEATS = ["hour","dow","month","hour_sin","hour_cos","dow_sin","dow_cos"]
FEATURES = (
    TOP_CORR_RAW +
    DIR_FEATS +
    miss_flags +
    TIME_FEATS +
    INTER_FEATS +
    Z_FEATS +
    lag_feats + roll_feats +
    DELTA_FEATS + EWM_FEATS +
    TARGET_LAGS
)

# Drop baris awal train yang masih NaN (efek lag/roll/ewm)
train_fe = train_fe.dropna(subset=FEATURES + [TARGET]).reset_index(drop=True)

y_raw = train_fe[TARGET].values
y = np.log1p(y_raw) if USE_LOG_TARGET else y_raw

X_all = train_fe[FEATURES].copy()
X_test = test_fe[FEATURES].copy()

# Isi NaN sisa test dengan median TRAIN (aman)
X_test = X_test.fillna(X_all.median())

print("n_features:", len(FEATURES))
print("train_X:", X_all.shape, "test_X:", X_test.shape, "| USE_LOG_TARGET:", USE_LOG_TARGET, "| USE_TARGET_LAGS:", USE_TARGET_LAGS)

# =========================================================
# Util — Standarisasi per fold (train-only stats)
# =========================================================
def standardize_fit_transform(X_tr, X_va=None):
    mu = X_tr.mean(axis=0)
    sigma = X_tr.std(axis=0, ddof=0).replace(0, 1.0)
    X_tr_std = (X_tr - mu) / sigma
    if X_va is None:
        return X_tr_std, mu, sigma, None
    X_va_std = (X_va - mu) / sigma
    return X_tr_std, mu, sigma, X_va_std

# =========================================================
# Blok 8 — TimeSeriesSplit CV dengan Linear vs Ridge (pilih terbaik)
# =========================================================
gap = steps_24h * GAP_MULTIPLIER
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

oof_lin = np.zeros(len(X_all))
oof_ridge = np.zeros(len(X_all)) if TRY_RIDGE else None
best_model_name_per_fold = []
fold = 0

test_preds_lin = []
test_preds_ridge = [] if TRY_RIDGE else None

for tr_idx, va_idx in tscv.split(X_all):
    fold += 1
    X_tr, y_tr = X_all.iloc[tr_idx], y[tr_idx]
    X_va, y_va = X_all.iloc[va_idx], y[va_idx]

    # Standarisasi (train-only)
    X_tr_std, mu, sigma, X_va_std = standardize_fit_transform(X_tr, X_va)

    # LinearRegression
    lin = LinearRegression(n_jobs=None)
    lin.fit(X_tr_std, y_tr)
    pred_va_lin = lin.predict(X_va_std)
    oof_lin[va_idx] = pred_va_lin

    # RidgeCV (opsional)
    if TRY_RIDGE:
        ridge = RidgeCV(alphas=RIDGE_ALPHAS, store_cv_values=False)
        ridge.fit(X_tr_std, y_tr)
        pred_va_ridge = ridge.predict(X_va_std)
        oof_ridge[va_idx] = pred_va_ridge

    # Evaluasi di ruang asli
    def to_raw(z):
        return np.clip(np.expm1(z) if USE_LOG_TARGET else z, 0, None)

    mse_lin = mean_squared_error(np.expm1(y_va) if USE_LOG_TARGET else y_va, to_raw(pred_va_lin))
    if TRY_RIDGE:
        mse_ridge = mean_squared_error(np.expm1(y_va) if USE_LOG_TARGET else y_va, to_raw(pred_va_ridge))
        if mse_ridge <= mse_lin:
            best_model_name_per_fold.append(("ridge", getattr(ridge, "alpha_", None)))
        else:
            best_model_name_per_fold.append(("linear", None))
    else:
        best_model_name_per_fold.append(("linear", None))

    # Predict test per model (pakai scaler fold)
    X_test_std = (X_test - mu) / sigma
    test_preds_lin.append(lin.predict(X_test_std))
    if TRY_RIDGE:
        test_preds_ridge.append(ridge.predict(X_test_std))

    print(f"Fold {fold} — MSE_lin={mse_lin:.6f}" + (f" | MSE_ridge={mse_ridge:.6f} (alpha~{getattr(ridge, 'alpha_', None)})" if TRY_RIDGE else ""))

# Pilih model berdasarkan OOF keseluruhan
def to_raw_arr(arr):
    return np.clip(np.expm1(arr) if USE_LOG_TARGET else arr, 0, None)

cv_mse_lin = mean_squared_error(y_raw, to_raw_arr(oof_lin))
print(f"\nCV MSE LinearRegression (OOF, raw) = {cv_mse_lin:.6f}")

if TRY_RIDGE:
    cv_mse_ridge = mean_squared_error(y_raw, to_raw_arr(oof_ridge))
    print(f"CV MSE RidgeCV (OOF, raw)         = {cv_mse_ridge:.6f}")
    use_ridge = (cv_mse_ridge <= cv_mse_lin)
else:
    use_ridge = False

print(">> Model terpilih:", "RidgeCV" if use_ridge else "LinearRegression")

# Blend test preds sesuai model terpilih
stack_te = np.vstack(test_preds_ridge if use_ridge else test_preds_lin)
pred_test_cvens_log = np.median(stack_te, axis=0)  # median fold → robust
pred_test_cvens = to_raw_arr(pred_test_cvens_log)

# =========================================================
# Blok 9 — Train Full Model di seluruh TRAIN (untuk submit_full)
# =========================================================
# Fit scaler full
X_all_std, mu_full, sigma_full, _ = standardize_fit_transform(X_all)

if use_ridge:
    final_model = RidgeCV(alphas=RIDGE_ALPHAS, store_cv_values=False)
else:
    final_model = LinearRegression()

final_model.fit(X_all_std, y)
X_test_std_full = (X_test - mu_full) / sigma_full
pred_test_full_log = final_model.predict(X_test_std_full)
pred_test_full = to_raw_arr(pred_test_full_log)

print("Pred test (full)  mean/std:", float(pred_test_full.mean()),  float(pred_test_full.std()))
print("Pred test (cvens) mean/std:", float(pred_test_cvens.mean()), float(pred_test_cvens.std()))

# =========================================================
# Blok 10 — Submission (pakai CV-median by default)
# =========================================================
FINAL_PRED = pred_test_cvens  # bisa ganti ke pred_test_full jika ingin

submission = pd.DataFrame({
    "Record number": test[IDCOL].values,
    "Turbidity": FINAL_PRED
})
# jaga urutan sample
if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))

# =========================================================
# Blok 11 — Artefak (model, scaler, OOF, koefisien)
# =========================================================
OUT_DIR = Path("./outputs_linear")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# simpan pilihan model & scaler stats
joblib.dump({"model": final_model, "mu": mu_full, "sigma": sigma_full, "features": FEATURES, "use_log_target": USE_LOG_TARGET}, OUT_DIR / "linear_full.pkl")

# submissions
submission.to_csv(OUT_DIR / "submission.csv", index=False)
pd.DataFrame({"Record number": test[IDCOL].values, "Turbidity": pred_test_full}).to_csv(
    OUT_DIR / "submission_full_model.csv", index=False
)
pd.DataFrame({"Record number": test[IDCOL].values, "Turbidity": pred_test_cvens}).to_csv(
    OUT_DIR / "submission_cv_ensemble.csv", index=False
)

# OOF simpan
oof_best = oof_ridge if use_ridge else oof_lin
oof_best_raw = to_raw_arr(oof_best)
oof_df = train_fe[[IDCOL, "Timestamp", TARGET]].copy()
oof_df["oof_pred"] = oof_best_raw[:len(oof_df)]
oof_df.to_csv(OUT_DIR / "oof_predictions.csv", index=False)

# Koefisien (jika tersedia)
coef_path = OUT_DIR / "coefficients.csv"
try:
    coefs = final_model.coef_.ravel() if hasattr(final_model, "coef_") else np.zeros(len(FEATURES))
    pd.DataFrame({"feature": FEATURES, "coefficient": coefs}).to_csv(coef_path, index=False)
    print("Top 20 |coef|:")
    print(pd.DataFrame({"feature": FEATURES, "coef_abs": np.abs(coefs)}).sort_values("coef_abs", ascending=False).head(20))
except Exception as e:
    print("Skip saving coefficients:", e)

# Metadata ringkas
meta = {
    "cv_mse_linear": float(cv_mse_lin),
    "cv_mse_ridge": float(cv_mse_ridge) if TRY_RIDGE else None,
    "model_chosen": "RidgeCV" if use_ridge else "LinearRegression",
    "n_splits": int(n_splits),
    "gap_steps": int(steps_24h * GAP_MULTIPLIER),
    "gap_hours": float((steps_24h * GAP_MULTIPLIER) / steps_per_hour),
    "steps_per_hour": int(steps_per_hour),
    "n_features": int(len(FEATURES)),
    "use_log_target": bool(USE_LOG_TARGET),
    "use_target_lags": bool(USE_TARGET_LAGS)
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ Artefak disimpan di:", OUT_DIR.resolve())


🧹 Dropped 6066 rows with NaN in 'Turbidity' from train.
steps/hour=6 | 1h=6 6h=36 12h=72 24h=144 48h=288
n_features: 262
train_X: (47346, 262) test_X: (14610, 262) | USE_LOG_TARGET: True | USE_TARGET_LAGS: False
Fold 1 — MSE_lin=1492.127336 | MSE_ridge=1136.228456 (alpha~0.09999999999999999)
Fold 2 — MSE_lin=52.318842 | MSE_ridge=52.304153 (alpha~0.0031622776601683794)
Fold 3 — MSE_lin=12.451476 | MSE_ridge=12.442850 (alpha~0.01)
Fold 4 — MSE_lin=11.219129 | MSE_ridge=11.210151 (alpha~0.09999999999999999)
Fold 5 — MSE_lin=497.478164 | MSE_ridge=501.076824 (alpha~0.09999999999999999)

CV MSE LinearRegression (OOF, raw) = 345.126921
CV MSE RidgeCV (OOF, raw)         = 286.404835
>> Model terpilih: RidgeCV
Pred test (full)  mean/std: 4.926139963461877 3.8387585812347904
Pred test (cvens) mean/std: 29.8006544378608 19.912590105090516
✅ submission.csv dibuat!
   Record number  Turbidity
0          54916  19.768608
1          54917  16.483177
2          54918  17.864685
3          54919  17.

In [ ]:
# =========================================================
# SARIMAX Turbidity — End-to-End with Exogenous
# =========================================================
import os, gc, math, json, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Statsmodels (SARIMAX)
try:
    from statsmodels.tsa.statespace.sarimax import SARIMAX
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "statsmodels"])
    from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_squared_error

# ===== Paths (Kaggle-style) =====
KAGGLE_DIR = Path("/kaggle/input/dataset-axion")
TRAIN_PATH = KAGGLE_DIR / "train.csv"
TEST_PATH  = KAGGLE_DIR / "test.csv"
SAMPLE_SUB_PATH = KAGGLE_DIR / "sample_submission.csv"

# Fallback lokal
if not TRAIN_PATH.exists():
    TRAIN_PATH = Path("./train.csv")
    TEST_PATH  = Path("./test.csv")
    SAMPLE_SUB_PATH = Path("./sample_submission.csv")

SEED = 42
np.random.seed(SEED)

TARGET = "Turbidity"
IDCOL  = "Record number"
DIR_COL = "Average Water Direction"  # degrees [0,360)

# Top-8 fitur sensor (raw)
TOP_CORR_RAW = [
    "Specific Conductance",
    "Average Water Speed",
    "Salinity",
    "Dissolved Oxygen (%Saturation)",
    "pH",
    "Dissolved Oxygen",
    "Temperature",
    "Chlorophyll",
]

# ---------- Opsi ----------
USE_LOG_TARGET      = True     # latih di log1p(y)
USE_DIR_FEATURES    = True     # tambah sin/cos & u,v dari arah arus
Q_LOW, Q_HIGH       = 0.005, 0.995   # capping quantiles (train-only)
VAL_FRAC            = 0.2      # validasi holdout 20% terakhir dari train
MAX_SEASON_S        = 240      # batasi s agar training tidak terlalu berat
PRINT_TOP_K_AIC     = 5        # tampilkan kandidat AIC terbaik

# =========================================================
# Blok 1 — Load & basic checks
# =========================================================
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SAMPLE_SUB_PATH)

for df in (train, test):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

train = train.sort_values("Timestamp").reset_index(drop=True)
test  = test.sort_values("Timestamp").reset_index(drop=True)

# Pastikan kolom minimal tersedia
need_train = TOP_CORR_RAW + [TARGET, "Timestamp", IDCOL]
need_test  = TOP_CORR_RAW + ["Timestamp", IDCOL]
for c in need_train:
    if c not in train.columns:
        print("⚠️ Missing train col:", c)
for c in need_test:
    if c not in test.columns:
        print("⚠️ Missing test col :", c)

# =========================================================
# Blok 1a — Drop target NaN
# =========================================================
train[TARGET] = pd.to_numeric(train[TARGET], errors="coerce")
n0 = len(train)
train = train[train[TARGET].notna()].reset_index(drop=True)
print(f"🧹 Dropped {n0-len(train)} rows with NaN target.")

# =========================================================
# Blok 2 — Estimasi resolusi waktu → steps/hour & s (24 jam)
# =========================================================
def estimate_steps_per_hour(df, time_col="Timestamp"):
    dt = df[time_col].diff().dt.total_seconds().dropna()
    if len(dt) == 0:
        return 1
    med = np.median(dt)
    if not np.isfinite(med) or med <= 0:
        return 1
    return int(max(1, round(3600.0/med)))

steps_per_hour = estimate_steps_per_hour(train)
steps_24h = max(1, 24 * steps_per_hour)
s_period  = steps_24h if steps_24h <= MAX_SEASON_S else None  # clamp seasonality if huge

print(f"steps/hour={steps_per_hour} | seasonal_s={s_period if s_period else 'None'}")

# =========================================================
# Blok 3 — Outlier capping (train-only bounds) untuk exog
# =========================================================
def compute_caps(ref_df, cols, q_low, q_high):
    bounds = {}
    for c in cols:
        lo, hi = ref_df[c].quantile([q_low, q_high])
        if not np.isfinite(lo): lo = ref_df[c].min()
        if not np.isfinite(hi): hi = ref_df[c].max()
        bounds[c] = (lo, hi)
    return bounds

def apply_caps(df, bounds):
    out = df.copy()
    for c, (lo, hi) in bounds.items():
        if c in out.columns:
            out[c] = out[c].clip(lo, hi)
    return out

caps = compute_caps(train, TOP_CORR_RAW, Q_LOW, Q_HIGH)
train = apply_caps(train, caps)
test  = apply_caps(test,  caps)

# =========================================================
# Blok 4 — Flag missingness & Imputasi time-aware (TRAIN & TEST terpisah)
# =========================================================
IMP_COLS = TOP_CORR_RAW + ([DIR_COL] if (USE_DIR_FEATURES and DIR_COL in train.columns) else [])

# pH & Direction sanitasi
train["pH"] = train["pH"].clip(0, 14)
test["pH"]  = test["pH"].clip(0, 14)
if USE_DIR_FEATURES and DIR_COL in train.columns:
    train[DIR_COL] = (train[DIR_COL] % 360).clip(0, 360)
if USE_DIR_FEATURES and DIR_COL in test.columns:
    test[DIR_COL]  = (test[DIR_COL]  % 360).clip(0, 360)

# median TRAIN-only
train_medians = {c: train[c].median(skipna=True) for c in IMP_COLS}

def impute_train_timeaware(df, cols, roll_win, ffill_limit, med_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        # rolling median masa lalu (shift 1)
        past_med = s.shift(1).rolling(window=roll_win, min_periods=1).median()
        s = s.where(~s.isna(), past_med)
        s = s.ffill(limit=ffill_limit)            # ≤ 6 jam
        s = s.fillna(med_map.get(c, s.median()))  # fallback median TRAIN
        out[c] = s
    return out

def impute_test_seeded(df, cols, ffill_limit, med_map, seed_map):
    out = df.copy()
    for c in cols:
        s = out[c].copy()
        if len(s)>0 and pd.isna(s.iloc[0]) and pd.notna(seed_map.get(c, np.nan)):
            s.iloc[0] = seed_map[c]
        s = s.ffill(limit=ffill_limit)
        s = s.fillna(med_map.get(c, s.median()))
        out[c] = s
    return out

# imputasi train
steps_6h = max(1, 6*steps_per_hour)
train_imp = impute_train_timeaware(train, IMP_COLS, roll_win=steps_6h, ffill_limit=steps_6h, med_map=train_medians)

# seed untuk test: last-known dari train imputasi
last_known_from_train = {c: (train_imp[c].iloc[-1] if pd.notna(train_imp[c].iloc[-1]) else train_medians[c]) for c in IMP_COLS}

# imputasi test
test_imp = impute_test_seeded(test, IMP_COLS, ffill_limit=steps_6h, med_map=train_medians, seed_map=last_known_from_train)

# =========================================================
# Blok 5 — Exogenous assembly (+ opsi fitur arah)
# =========================================================
def add_dir_features(df):
    if (DIR_COL in df.columns):
        rad = np.deg2rad((df[DIR_COL] % 360).fillna(0.0))
        dir_sin = np.sin(rad)
        dir_cos = np.cos(rad)
        u = df["Average Water Speed"] * dir_cos
        v = df["Average Water Speed"] * dir_sin
        return pd.concat([df, 
                          pd.DataFrame({"dir_sin":dir_sin, "dir_cos":dir_cos, "u":u, "v":v}, index=df.index)
                         ], axis=1)
    return df

train_imp = add_dir_features(train_imp.copy())
test_imp  = add_dir_features(test_imp.copy())

EXOG_COLS = TOP_CORR_RAW + (["dir_sin","dir_cos","u","v"] if (USE_DIR_FEATURES and "u" in train_imp.columns) else [])

# Standarisasi exog pakai TRAIN-only
mu_exog = train_imp[EXOG_COLS].mean(axis=0)
sd_exog = train_imp[EXOG_COLS].std(axis=0).replace(0, 1.0)

X_train = (train_imp[EXOG_COLS] - mu_exog) / sd_exog
X_test  = (test_imp[EXOG_COLS]  - mu_exog) / sd_exog

# =========================================================
# Blok 6 — Target transform (opsional)
# =========================================================
y_train_raw = train[TARGET].values
y_train = np.log1p(y_train_raw) if USE_LOG_TARGET else y_train_raw

# Index waktu agar stabil
y_index = pd.Series(y_train, index=train_imp["Timestamp"])
X_train.index = train_imp["Timestamp"]
X_test.index  = test_imp["Timestamp"]

print("Exog cols:", len(EXOG_COLS), "| Train:", X_train.shape, "| Test:", X_test.shape, "| LogTarget:", USE_LOG_TARGET)

# =========================================================
# Blok 7 — Kandidat SARIMAX & validasi holdout
# =========================================================
# Seasonal period s
use_season = (s_period is not None and s_period >= 2)

# Kandidat non-seasonal (p,d,q)
pdq_list = [(1,1,0), (0,1,1), (1,1,1), (2,1,1)]

# Kandidat seasonal (P,D,Q,s) — jika dipakai
seasonal_list = [(0,0,0,0)]
if use_season:
    seasonal_list += [(1,0,1,s_period), (0,1,1,s_period)]

# Split holdout
n = len(y_index)
split = int(max(1, round((1.0 - VAL_FRAC) * n)))
y_tr, y_va = y_index.iloc[:split], y_index.iloc[split:]
X_tr, X_va = X_train.iloc[:split], X_train.iloc[split:]

def safe_fit(pdq, sdq):
    try:
        if sdq[3] == 0:  # no seasonality
            seasonal_order = (0,0,0,0)
        else:
            seasonal_order = sdq
        model = SARIMAX(
            y_tr,
            exog=X_tr,
            order=pdq,
            seasonal_order=seasonal_order,
            trend="c",
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        res = model.fit(disp=False, maxiter=200)
        # forecast pada validasi
        fc = res.get_forecast(steps=len(y_va), exog=X_va).predicted_mean
        # inverse & clip
        pred_va = np.expm1(fc.values) if USE_LOG_TARGET else fc.values
        pred_va = np.clip(pred_va, 0, None)
        y_va_raw = np.expm1(y_va.values) if USE_LOG_TARGET else y_va.values
        mse = mean_squared_error(y_va_raw, pred_va)
        aic = res.aic
        return {"pdq": pdq, "sdq": seasonal_order, "mse": mse, "aic": aic, "res": res}
    except Exception as e:
        return None

results = []
for pdq in pdq_list:
    for sdq in seasonal_list:
        out = safe_fit(pdq, sdq)
        if out is not None:
            results.append(out)
        print(f"try pdq={pdq}, sdq={sdq} =>", "OK" if out else "fail")

if len(results) == 0:
    raise RuntimeError("Semua kandidat SARIMAX gagal fit. Coba kecilkan dimensi exog/ matikan seasonality.")

# rangkum & pilih terbaik (berdasarkan MSE validasi)
results_sorted = sorted(results, key=lambda d: (d["mse"], d["aic"]))
print("\nTop candidates (by MSE):")
for r in results_sorted[:PRINT_TOP_K_AIC]:
    print(f"  pdq={r['pdq']}, sdq={r['sdq']}, MSE={r['mse']:.4f}, AIC={r['aic']:.1f}")

best = results_sorted[0]
print(f"\n✅ Best (holdout): pdq={best['pdq']}, sdq={best['sdq']}, MSE_val={best['mse']:.6f}, AIC={best['aic']:.1f}")

# =========================================================
# Blok 8 — Refit di FULL TRAIN & Forecast TEST
# =========================================================
def fit_full_and_forecast(pdq, sdq):
    model = SARIMAX(
        y_index,             # full train (log/linear sesuai USE_LOG_TARGET)
        exog=X_train,
        order=pdq,
        seasonal_order=sdq,
        trend="c",
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    res = model.fit(disp=False, maxiter=300)
    fc_test = res.get_forecast(steps=len(X_test), exog=X_test).predicted_mean
    pred_test = np.expm1(fc_test.values) if USE_LOG_TARGET else fc_test.values
    pred_test = np.clip(pred_test, 0, None)
    return res, pred_test

res_full, pred_test = fit_full_and_forecast(best["pdq"], best["sdq"])

print("Pred test mean/std:", float(np.mean(pred_test)), float(np.std(pred_test)))

# =========================================================
# Blok 9 — Submission
# =========================================================
submission = pd.DataFrame({
    "Record number": test[IDCOL].values,
    "Turbidity": pred_test
})
# jaga urutan sample
if "Record number" in sub.columns:
    submission = sub[["Record number"]].merge(submission, on="Record number", how="left")

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv dibuat!")
print(submission.head(10))

# =========================================================
# Blok 10 — Artefak & Metadata
# =========================================================
OUT_DIR = Path("./outputs_sarimax")
OUT_DIR.mkdir(parents=True, exist_ok=True)

submission.to_csv(OUT_DIR / "submission.csv", index=False)

# Simpan ringkas parameter & statistik
meta = {
    "use_log_target": bool(USE_LOG_TARGET),
    "exog_cols": EXOG_COLS,
    "n_exog": int(len(EXOG_COLS)),
    "steps_per_hour": int(steps_per_hour),
    "seasonal_period": int(s_period) if s_period else None,
    "best_pdq": list(best["pdq"]),
    "best_sdq": list(best["sdq"]),
    "val_mse": float(best["mse"]),
    "aic": float(best["aic"]),
    "caps_quantiles": [Q_LOW, Q_HIGH],
    "caps_bounds": {k: [float(v[0]), float(v[1])] for k, v in caps.items()}
}
with open(OUT_DIR / "metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

# simpan exog scaler
pd.Series(EXOG_COLS).to_csv(OUT_DIR / "exog_features.txt", index=False, header=False)
pd.concat([mu_exog.rename("mu"), sd_exog.rename("sigma")], axis=1).to_csv(OUT_DIR / "exog_scaler.csv")

print("📦 Artefak disimpan di:", OUT_DIR.resolve())


🧹 Dropped 6066 rows with NaN target.
steps/hour=6 | seasonal_s=144
Exog cols: 12 | Train: (47382, 12) | Test: (14610, 12) | LogTarget: True


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


try pdq=(1, 1, 0), sdq=(0, 0, 0, 0) => OK


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
# ===== Tambahan: set frekuensi waktu agar SARIMAX tidak warning =====
import pandas as pd

def freq_from_steps_per_hour(sph: int) -> str:
    step_sec = int(round(3600 / max(1, sph)))
    if step_sec % 60 == 0:
        return f"{step_sec // 60}T"   # menit
    return f"{step_sec}S"             # detik

# Coba infer dari data dulu; jika gagal, fallback dari steps/hour
freq_infer = pd.infer_freq(train_imp["Timestamp"])
freq = freq_infer if freq_infer is not None else freq_from_steps_per_hour(steps_per_hour)
print("Using freq:", freq)

# Seri target dengan index waktu
y_series = pd.Series(y_train, index=train_imp["Timestamp"]).asfreq(freq)

# Exog: pastikan index waktu & freq sama, lalu reindex ke target
X_train = X_train.copy()
X_test  = X_test.copy()
X_train.index = train_imp["Timestamp"]
X_test.index  = test_imp["Timestamp"]

X_train = X_train.asfreq(freq)
X_test  = X_test.asfreq(freq)

# Selaraskan exog ke index target; isi potensi celah exog dengan ffill/bfill lokal
X_train = X_train.reindex(y_series.index).ffill().bfill()

# Buang baris target yang NaN (jika asfreq menambah grid kosong di tengah)
valid_idx = y_series.dropna().index
y_index = y_series.loc[valid_idx]
X_train = X_train.loc[valid_idx]

# (opsional) pastikan test exog juga rapih
X_test = X_test.ffill().bfill()
